# IMPORT LIBRARIES AND FUNCTIONS

In [ ]:
!pip install pyod
!pip install combo
!pip install suod
!pip install pynomaly

In [ ]:
%matplotlib inline
import math
import numpy as np
import pandas as pd 
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import random 
import time
from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report, roc_curve, auc, plot_confusion_matrix, roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.cluster import DBSCAN
from PyNomaly import loop

from sklearn.covariance import EllipticEnvelope

from sklearn.neighbors import LocalOutlierFactor

# evaluate model performance with outliers removed using isolation forest
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error

from pyod.models.knn import KNN   # kNN detector
import pyod
from pyod.utils.data import generate_data
from pyod.utils.utility import score_to_label

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# IMPORT DATA

In [ ]:
directory = '/content/drive/My Drive/TESI_LUSITO/benchmark_data/yeast/'
dataname = 'yeast_clean.csv'

In [ ]:
data = pd.read_csv(directory+dataname, delimiter=',') #load data
data.drop('Unnamed: 0', inplace=True, axis=1)
#data.drop('outlier', inplace=True, axis=1)
data.head()

In [ ]:
target = data['class']
data.drop('class', inplace=True, axis=1) 

#supervised?
#data.drop('outlier', inplace=True, axis=1)

#OUTLIERNESS SCORES EXTRACTION ON ORIGINAL DATA

##pynomaly

In [ ]:
scores_df = pd.DataFrame()
binary_scores_df = pd.DataFrame()

In [ ]:
def pynomaly_methods(data, scores_df, binary_scores_df):
  #no parameters to test
  try:
    m = loop.LocalOutlierProbability(data, progress_bar=True).fit()
    scores = m.local_outlier_probabilities
    scores_df['pynomaly_loop_original'] = scores
    binary_scores_df['pynomaly_loop_original_binary'] = scores
    binary_scores_df['pynomaly_loop_original_binary'] = np.where(binary_scores_df['pynomaly_loop_original_binary']>=0.5, 1, 0)
  except:
    print('Exception Raised --> LOP Config 1')

  #parameters to test:
  # -eps/min_samples for DBSCAN
  # -extend/n_neighbors for LOP

  try:
    db = DBSCAN(eps=0.1, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps01_sam10_nei10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam10_nei10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam10_nei10_e1'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps01_sam10_nei10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 1')

  try:
    db = DBSCAN(eps=0.1, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps01_sam20_nei10_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam20_nei10_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam20_nei10_e2'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps01_sam20_nei10_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 2')

  try:  
    db = DBSCAN(eps=0.1, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, n_neighbors=20, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps01_sam20_nei20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam20_nei20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam20_nei20_e2'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps01_sam20_nei20_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 3')

  try:  
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps01_sam50_nei10_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei10_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei10_e3'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei10_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 4')

  try: 
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=20, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps01_sam50_nei20_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei20_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei20_e3'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei20_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 5')

  try:  
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=50, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps01_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei50_e3'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps01_sam50_nei50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 6')

  try:
    db = DBSCAN(eps=0.3, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilitiesscores_df['dbscan_scores_original_eps03_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam10_e1'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps03_sam10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 7')

  try:
    db = DBSCAN(eps=0.3, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps03_sam20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam20_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam20_e1'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps03_sam20_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 8')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2,  cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps03_sam50_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e2'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 9')

  try:  
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3,cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e3'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 10')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_original_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e3'] = np.where(binary_scores_df['dbscan_scores_original_binary_eps03_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 11')

##sklearn algorithms

In [ ]:
def sklearn_methods(data, scores_df, binary_scores_df):
  #Minimum Covariance Determinant
  try:
    ee = EllipticEnvelope(contamination=0.001)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_original_cont0.001'] = dec_func
    scores_df['elliptic_mahalanobis_original_cont0.001'] = maha
    binary_scores_df['elliptic_env_original_binary_cont0.001'] = pred
    binary_scores_df['elliptic_env_original_binary_cont0.001'] = np.where(binary_scores_df['elliptic_env_original_binary_cont0.001']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 1')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.01)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_original_cont0.01'] = dec_func
    scores_df['elliptic_mahalanobis_original_cont0.01'] = maha
    binary_scores_df['elliptic_env_original_binary_cont0.01'] = pred
    binary_scores_df['elliptic_env_original_binary_cont0.01'] = np.where(binary_scores_df['elliptic_env_original_binary_cont0.01']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 2')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.1)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_original_cont0.1'] = dec_func
    scores_df['elliptic_mahalanobis_original_cont0.1'] = maha
    binary_scores_df['elliptic_env_original_binary_cont0.1'] = pred
    binary_scores_df['elliptic_env_original_binary_cont0.1'] = np.where(binary_scores_df['elliptic_env_original_binary_cont0.1']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 3')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.2)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_original_cont0.2'] = dec_func
    scores_df['elliptic_mahalanobis_original_cont0.2'] = maha
    binary_scores_df['elliptic_env_original_binary_cont0.2'] = pred
    binary_scores_df['elliptic_env_original_binary_cont0.2'] = np.where(binary_scores_df['elliptic_env_original_binary_cont0.2']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 4')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.5)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_original_cont0.5'] = dec_func
    scores_df['elliptic_mahalanobis_original_cont0.5'] = maha
    binary_scores_df['elliptic_env_original_binary_cont0.5'] = pred
    binary_scores_df['elliptic_env_original_binary_cont0.5'] = np.where(binary_scores_df['elliptic_env_original_binary_cont0.5']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 5')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.001, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_original_cont0.001'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.001'] = pred
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.001'] = np.where(binary_scores_df['iso_forest_paper_score_original_binary_cont0.001']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 1')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.01, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_original_cont0.01'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.01'] = pred
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.01'] = np.where(binary_scores_df['iso_forest_paper_score_original_binary_cont0.01']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 2')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.1, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_original_cont0.1'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.1'] = pred
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.1'] = np.where(binary_scores_df['iso_forest_paper_score_original_binary_cont0.1']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 3')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.2, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_original_cont0.2'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.2'] = pred
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.2'] = np.where(binary_scores_df['iso_forest_paper_score_original_binary_cont0.2']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 4')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.5, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_original_cont0.5'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.5'] = pred
    binary_scores_df['iso_forest_paper_score_original_binary_cont0.5'] = np.where(binary_scores_df['iso_forest_paper_score_original_binary_cont0.5']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 5')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=10, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['lof_score_original_mink_nei10'] = lof_score
    binary_scores_df['lof_score_original_binary_mink_nei10'] = pred
    binary_scores_df['lof_score_original_binary_mink_nei10'] = np.where(binary_scores_df['lof_score_original_binary_mink_nei10']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 1')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=20, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['lof_score_original_mink_nei20'] = lof_score
    binary_scores_df['lof_score_original_binary_mink_nei20'] = pred
    binary_scores_df['lof_score_original_binary_mink_nei20'] = np.where(binary_scores_df['lof_score_original_binary_mink_nei20']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 2')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=50, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['lof_score_original_mink_nei50'] = lof_score
    binary_scores_df['lof_score_original_binary_mink_nei50'] = pred
    binary_scores_df['lof_score_original_binary_mink_nei50'] = np.where(binary_scores_df['lof_score_original_binary_mink_nei50']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 3')

  from sklearn.svm import OneClassSVM
  # try:
  #   #One Class SVM
  #   ee = OneClassSVM(nu=0.001, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_original_linear_nu0.001'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_original_binary_linear_nu0.001'] = pred
  #   binary_scores_df['oneclass_SVM_original_binary_linear_nu0.001'] = np.where(binary_scores_df['oneclass_SVM_original_binary_linear_nu0.001']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 1')

  # try:
  #   ee = OneClassSVM(nu=0.01, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_original_linear_nu0.01'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_original_binary_linear_nu0.01'] = pred
  #   binary_scores_df['oneclass_SVM_original_binary_linear_nu0.01'] = np.where(binary_scores_df['oneclass_SVM_original_binary_linear_nu0.01']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 2')

  # try:
  #   ee = OneClassSVM(nu=0.1, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_original_linear_nu0.1'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_original_binary_linear_nu0.1'] = pred
  #   binary_scores_df['oneclass_SVM_original_binary_linear_nu0.1'] = np.where(binary_scores_df['oneclass_SVM_original_binary_linear_nu0.1']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 3')

  # try:
  #   ee = OneClassSVM(nu=0.001)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_original_rbf_nu0.001'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.001'] = pred
  #   binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.001'] = np.where(binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.001']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 4')

  # try:  
  #   ee = OneClassSVM(nu=0.01)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_original_rbf_nu0.01'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.01'] = pred
  #   binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.01'] = np.where(binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.01']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 5')

  # try:
  #   ee = OneClassSVM(nu=0.1)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_original_rbf_nu0.1'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.1'] = pred
  #   binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.1'] = np.where(binary_scores_df['oneclass_SVM_original_binary_rbf_nu0.1']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 6')

##pyod

In [ ]:
from pyod.models.mcd import MCD
from pyod.models.suod import SUOD
from pyod.models.loda import LODA
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.cblof import CBLOF
from pyod.models.ocsvm import OCSVM
from pyod.models.cof import COF
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
from pyod.models.sos import SOS
from pyod.models.pca import PCA

def pyod_methods(data, scores_df, binary_scores_df):
  try:
    clf = COPOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_original_cont0.001'] = pyod_copod
    binary_scores_df['pyod_copod_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> COPOD Config 1')

  try:
    clf = COPOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_original_cont0.01'] = pyod_copod
    binary_scores_df['pyod_copod_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> COPOD Config 2')

  try:
    clf = COPOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_original_cont0.1'] = pyod_copod
    binary_scores_df['pyod_copod_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> COPOD Config 3')

  try:
    clf = COPOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_original_cont0.2'] = pyod_copod
    binary_scores_df['pyod_copod_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> COPOD Config 4')

  try:
    clf = COPOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_original_cont0.5'] = pyod_copod
    binary_scores_df['pyod_copod_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> COPOD Config 5')

  try:
    clf = ECOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_original_cont0.001'] = pyod_ecod
    binary_scores_df['pyod_ecod_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> ECOD Config 1')

  try:
    clf = ECOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_original_cont0.01'] = pyod_ecod
    binary_scores_df['pyod_ecod_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> ECOD Config 2')

  try:
    clf = ECOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_original_cont0.1'] = pyod_ecod
    binary_scores_df['pyod_ecod_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> ECOD Config 3')

  try:
    clf = ECOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_original_cont0.2'] = pyod_ecod
    binary_scores_df['pyod_ecod_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> ECOD Config 4')

  try:
    clf = ECOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_original_cont0.5'] = pyod_ecod
    binary_scores_df['pyod_ecod_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> ECOD Config 5')

  try:
    clf = SOS(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_original_cont0.001'] = pyod_sos
    binary_scores_df['pyod_sos_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> SOS Config 1')

  try:
    clf = SOS(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_original_cont0.01'] = pyod_sos
    binary_scores_df['pyod_sos_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> SOS Config 2')

  try:
    clf = SOS(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_original_cont0.1'] = pyod_sos
    binary_scores_df['pyod_sos_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> SOS Config 3')

  try:
    clf = SOS(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_original_cont0.2'] = pyod_sos
    binary_scores_df['pyod_sos_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> SOS Config 4')

  try:
    clf = SOS(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_original_cont0.5'] = pyod_sos
    binary_scores_df['pyod_sos_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> SOS Config 5')

  try:
    clf = PCA(n_components=2, contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.001_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.001_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 1 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.01_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.01_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 2 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.1_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.1_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 3 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.2_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.2_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 4 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.5_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.5_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 5 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.001_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.001_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 6 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.01_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.01_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 7 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.1_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.1_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 8 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.2_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.2_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 9 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_original_cont0.5_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_original_binary_cont0.5_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 10 data maybe not converged')

  try:
    clf = MCD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_original_cont0.001'] = pyod_mcd
    binary_scores_df['pyod_mcd_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> MCD Config 1')

  try:
    clf = MCD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_original_cont0.01'] = pyod_mcd
    binary_scores_df['pyod_mcd_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> MCD Config 2')

  try:
    clf = MCD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_original_cont0.1'] = pyod_mcd
    binary_scores_df['pyod_mcd_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> MCD Config 3')

  try:
    clf = MCD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_original_cont0.2'] = pyod_mcd
    binary_scores_df['pyod_mcd_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> MCD Config 4')

  try:
    clf = MCD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_original_cont0.5'] = pyod_mcd
    binary_scores_df['pyod_mcd_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> MCD Config 5')

  try:
    clf = OCSVM(contamination=0.001, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.001_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 1')

  try:
    clf = OCSVM(contamination=0.01, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.01_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.01_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 2')

  try:
    clf = OCSVM(contamination=0.1, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.1_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.1_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 3')

  try:
    clf = OCSVM(contamination=0.2, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.2_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.2_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 4')

  try:
    clf = OCSVM(contamination=0.5, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.5_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.5_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 5')

  try:
    clf = OCSVM(contamination=0.001, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.001_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 6')

  try:
    clf = OCSVM(contamination=0.001, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.001_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 7')

  try:
    clf = OCSVM(contamination=0.001, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.001_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 8')

  try:
    clf = OCSVM(contamination=0.01, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.01_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 9')

  try:
    clf = OCSVM(contamination=0.01, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.01_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 10')

  try:
    clf = OCSVM(contamination=0.01, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.01_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.001_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 11')

  try:
    clf = OCSVM(contamination=0.1, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.1_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.1_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 12')

  try:
    clf = OCSVM(contamination=0.1, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.1_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.1_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 13')

  try:
    clf = OCSVM(contamination=0.1, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.1_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.1_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 14')

  try:
    clf = OCSVM(contamination=0.2, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.2_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.2_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 15')

  try:
    clf = OCSVM(contamination=0.2, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.2_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.2_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 16')

  try:
    clf = OCSVM(contamination=0.2, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.2_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.2_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 17')

  try:
    clf = OCSVM(contamination=0.5, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.5_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.5_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 18')

  try:
    clf = OCSVM(contamination=0.5, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.5_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.5_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 19')

  try:
    clf = OCSVM(contamination=0.5, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_original_cont0.5_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_original_binary_cont0.5_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 20')

  try:
    clf = COF(contamination=0.001, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.001_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.001_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 1')

  try:
    clf = COF(contamination=0.01, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.01_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.01_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 2')

  try:
    clf = COF(contamination=0.1, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.1_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.1_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 3')

  try:
    clf = COF(contamination=0.2, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.2_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.2_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 4')

  try:
    clf = COF(contamination=0.5, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.5_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.5_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 5')

  try:
    clf = COF(contamination=0.001, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.001_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.001_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 6')

  try:
    clf = COF(contamination=0.01, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.01_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.01_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 7')

  try:
    clf = COF(contamination=0.1, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.1_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.1_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 8')

  try:
    clf = COF(contamination=0.2, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.2_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.2_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 9')

  try:
    clf = COF(contamination=0.5, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.5_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.5_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 10')

  try:
    clf = COF(contamination=0.001, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.001_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.001_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 11')

  try:
    clf = COF(contamination=0.01, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.01_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.01_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 12')

  try:
    clf = COF(contamination=0.1, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.1_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.1_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 13')

  try:
    clf = COF(contamination=0.2, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.2_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.2_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 14')

  try:
    clf = COF(contamination=0.5, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_original_cont0.5_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_original_binary_cont0.5_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 15')

  try:
    clf = CBLOF(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_original_cont0.001'] = pyod_cblof
    binary_scores_df['pyod_cblof_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> CBLOF Config 1')

  try:  
    clf = CBLOF(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_original_cont0.01'] = pyod_cblof
    binary_scores_df['pyod_cblof_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> CBLOF Config 2')

  try:
    clf = CBLOF(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_original_cont0.1'] = pyod_cblof
    binary_scores_df['pyod_cblof_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> CBLOF Config 3')

  try:
    clf = CBLOF(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_original_cont0.2'] = pyod_cblof
    binary_scores_df['pyod_cblof_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> CBLOF Config 4')

  try:
    clf = CBLOF(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_original_cont0.5'] = pyod_cblof
    binary_scores_df['pyod_cblof_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> CBLOF Config 5')

  try:
    clf = HBOS(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_original_cont0.001'] = pyod_hbos
    binary_scores_df['pyod_hbos_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> HBOS Config 1')

  try:
    clf = HBOS(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_original_cont0.01'] = pyod_hbos
    binary_scores_df['pyod_hbos_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> HBOS Config 2')

  try:
    clf = HBOS(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_original_cont0.1'] = pyod_hbos
    binary_scores_df['pyod_hbos_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> HBOS Config 3')

  try:
    clf = HBOS(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_original_cont0.2'] = pyod_hbos
    binary_scores_df['pyod_hbos_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> HBOS Config 4')

  try:
    clf = HBOS(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_original_cont0.5'] = pyod_hbos
    binary_scores_df['pyod_hbos_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> HBOS Config 5')

  try:
    clf = KNN(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_original_cont0.001'] = pyod_knn
    binary_scores_df['pyod_knn_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> KNN Config 1')

  try:
    clf = KNN(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_original_cont0.01'] = pyod_knn
    binary_scores_df['pyod_knn_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> KNN Config 2')

  try:  
    clf = KNN(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_original_cont0.1'] = pyod_knn
    binary_scores_df['pyod_knn_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> KNN Config 3')

  try:
    clf = KNN(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_original_cont0.2'] = pyod_knn
    binary_scores_df['pyod_knn_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> KNN Config 4')

  try:
    clf = KNN(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_original_cont0.5'] = pyod_knn
    binary_scores_df['pyod_knn_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> KNN Config 5')

  try:
    clf = FeatureBagging(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_original_cont0.001'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 1')

  try:
    clf = FeatureBagging(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_original_cont0.01'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 2')

  try:
    clf = FeatureBagging(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_original_cont0.1'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 3')

  try:
    clf = FeatureBagging(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_original_cont0.2'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 4')

  try:
    clf = FeatureBagging(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_original_cont0.5'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 5')

  try:
    clf = LODA(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_original_cont0.001'] = pyod_loda
    binary_scores_df['pyod_loda_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> LODA Config 1')

  try:
    clf = LODA(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_original_cont0.01'] = pyod_loda
    binary_scores_df['pyod_loda_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> LODA Config 2')

  try:
    clf = LODA(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_original_cont0.1'] = pyod_loda
    binary_scores_df['pyod_loda_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> LODA Config 3')

  try:
    clf = LODA(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_original_cont0.2'] = pyod_loda
    binary_scores_df['pyod_loda_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> LODA Config 4')

  try:
    clf = LODA(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_original_cont0.5'] = pyod_loda
    binary_scores_df['pyod_loda_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> LODA Config 5')

  try:
    clf = SUOD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_original_cont0.001'] = pyod_suod
    binary_scores_df['pyod_suod_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> SUOD Config 1')

  try:
    clf = SUOD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_original_cont0.01'] = pyod_suod
    binary_scores_df['pyod_suod_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> SUOD Config 2')

  try:
    clf = SUOD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_original_cont0.1'] = pyod_suod
    binary_scores_df['pyod_suod_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> SUOD Config 3')

  try:
    clf = SUOD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_original_cont0.2'] = pyod_suod
    binary_scores_df['pyod_suod_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> SUOD Config 4')

  try:
    clf = SUOD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_original_cont0.5'] = pyod_suod
    binary_scores_df['pyod_suod_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> SUOD Config 5')

##network methods

In [ ]:
#autoencoder method - (improve network)
from pyod.models.auto_encoder import AutoEncoder
def network_methods(data):
  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_original_cont0.001'] = pyod_encoder
    binary_scores_df['pyod_encoder_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_original_cont0.01'] = pyod_encoder
    binary_scores_df['pyod_encoder_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_original_cont0.1'] = pyod_encoder
    binary_scores_df['pyod_encoder_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_original_cont0.2'] = pyod_encoder
    binary_scores_df['pyod_encoder_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_original_cont0.5'] = pyod_encoder
    binary_scores_df['pyod_encoder_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  #vae method
  from pyod.models.vae import VAE

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_original_cont0.001'] = pyod_vae
    binary_scores_df['pyod_vae_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_original_cont0.01'] = pyod_vae
    binary_scores_df['pyod_vae_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_original_cont0.1'] = pyod_vae
    binary_scores_df['pyod_vae_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_original_cont0.2'] = pyod_vae
    binary_scores_df['pyod_vae_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_original_cont0.5'] = pyod_vae
    binary_scores_df['pyod_vae_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> VAE Config')

  #so_gaal method
  from pyod.models.so_gaal import SO_GAAL

  try:
    clf = SO_GAAL(contamination=0.001, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.001_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.001_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.01, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.01_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.01_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.1_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.1_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.2, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.2_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.2_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.5, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.5_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.5_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.1_lrd0.01_lrg0.01_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.1_lrd0.01_lrg0.01_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.3 , lr_g=0.3 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.1_lrd0.3_lrg0.3_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.1_lrd0.3_lrg0.3_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.1_lrd0.01_lrg0.01_mom0.1'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.1_lrd0.01_lrg0.01_mom0.1'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_original_cont0.1_lrd0.01_lrg0.01_mom0.3'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_original_binary_cont0.1_lrd0.01_lrg0.01_mom0.3'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  #deepsvdd method
  import pyod.models.deep_svdd
  try:
    clf = pyod.models.deep_svdd.DeepSVDD(verbose=1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd = clf.decision_scores_
    scores_df['pyod_svdd_original'] = pyod_svdd
    binary_scores_df['pyod_svdd_original_binary'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  #mo_gaal method
  from pyod.models.mo_gaal import MO_GAAL

  try:
    clf = MO_GAAL(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_original_cont0.001'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_original_cont0.01'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_original_cont0.1'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_original_cont0.2'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_original_cont0.5'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  #deep_svd method
  from pyod.models.deep_svdd import DeepSVDD

  try:
    clf = DeepSVDD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_original_cont0.001'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_original_cont0.01'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_original_cont0.1'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_original_cont0.2'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_original_cont0.5'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

#DIMENSIONALITY REDUCTION TECHNIQUES ON ORIGINAL DATA



In [ ]:
def sammon(x, n, display = 0, inputdist = 'raw', maxhalves = 20, maxiter = 1, tolfun = 1e-9, init = 'default'):

    import numpy as np 
    from scipy.spatial.distance import cdist

    """Perform Sammon mapping on dataset x
    y = sammon(x) applies the Sammon nonlinear mapping procedure on
    multivariate data x, where each row represents a pattern and each column
    represents a feature.  On completion, y contains the corresponding
    co-ordinates of each point on the map.  By default, a two-dimensional
    map is created.  Note if x contains any duplicated rows, SAMMON will
    fail (ungracefully). 
    [y,E] = sammon(x) also returns the value of the cost function in E (i.e.
    the ess of the mapping).
    An N-dimensional output map is generated by y = sammon(x,n) .
    A set of optimisation options can be specified using optional
    arguments, y = sammon(x,n,[OPTS]):
       maxiter        - maximum number of iterations
       tolfun         - relative tolerance on objective function
       maxhalves      - maximum number of step halvings
       input          - {'raw','distance'} if set to 'distance', X is 
                        interpreted as a matrix of pairwise distances.
       display        - 0 to 2. 0 least verbose, 2 max verbose.
       init           - {'pca', 'cmdscale', random', 'default'}
                        default is 'pca' if input is 'raw', 
                        'msdcale' if input is 'distance'
    The default options are retrieved by calling sammon(x) with no
    parameters.
    File        : sammon.py
    Date        : 18 April 2014
    Authors     : Tom J. Pollard (tom.pollard.11@ucl.ac.uk)
                : Ported from MATLAB implementation by 
                  Gavin C. Cawley and Nicola L. C. Talbot
    Description : Simple python implementation of Sammon's non-linear
                  mapping algorithm [1].
    References  : [1] Sammon, John W. Jr., "A Nonlinear Mapping for Data
                  Structure Analysis", IEEE Transactions on Computers,
                  vol. C-18, no. 5, pp 401-409, May 1969.
    Copyright   : (c) Dr Gavin C. Cawley, November 2007.
    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program; if not, write to the Free Software
    Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
    """

    # Create distance matrix unless given by parameters
    if inputdist == 'distance':
        D = x
        if init == 'default':
            init = 'cmdscale'
    else:
        D = cdist(x, x)
        if init == 'default':
            init = 'pca'

    if inputdist == 'distance' and init == 'pca':
        raise ValueError("Cannot use init == 'pca' when inputdist == 'distance'")

    if np.count_nonzero(np.diagonal(D)) > 0:
        raise ValueError("The diagonal of the dissimilarity matrix must be zero")

    # Remaining initialisation
    N = x.shape[0]
    scale = 0.5 / D.sum()
    D = D + np.eye(N)     

    if np.count_nonzero(D<=0) > 0:
        raise ValueError("Off-diagonal dissimilarities must be strictly positive")   

    Dinv = 1 / D
    if init == 'pca':
        [UU,DD,_] = np.linalg.svd(x)
        y = UU[:,:n]*DD[:n] 
    elif init == 'cmdscale':
        from cmdscale import cmdscale
        y,e = cmdscale(D)
        y = y[:,:n]
    else:
        y = np.random.normal(0.0,1.0,[N,n])
    one = np.ones([N,n])
    d = cdist(y,y) + np.eye(N)
    dinv = 1. / d
    delta = D-d 
    E = ((delta**2)*Dinv).sum() 

    # Get on with it
    for i in range(maxiter):
        # Compute gradient, Hessian and search direction (note it is actually
        # 1/4 of the gradient and Hessian, but the step size is just the ratio
        # of the gradient and the diagonal of the Hessian so it doesn't
        # matter).
        delta = dinv - Dinv
        deltaone = np.dot(delta,one)
        g = np.dot(delta,y) - (y * deltaone)
        dinv3 = dinv ** 3
        y2 = y ** 2
        H = np.dot(dinv3,y2) - deltaone - np.dot(2,y) * np.dot(dinv3,y) + y2 * np.dot(dinv3,one)
        s = -g.flatten(order='F') / np.abs(H.flatten(order='F'))
        y_old    = y

        # Use step-halving procedure to ensure progress is made
        for j in range(maxhalves):
            s_reshape = np.reshape(s, (-1,n),order='F')
            y = y_old + s_reshape
            d = cdist(y, y) + np.eye(N)
            dinv = 1 / d
            delta = D - d
            E_new = ((delta**2)*Dinv).sum()
            if E_new < E:
                break
            else:
                s = 0.5*s

        # Bomb out if too many halving steps are required
        if j == maxhalves-1:
            print('Warning: maxhalves exceeded. Sammon mapping may not converge...')

        # Evaluate termination criterion
        if abs((E - E_new) / E) < tolfun:
            if display:
                print('TolFun exceeded: Optimisation terminated')
            break

        # Report progress
        E = E_new
        if display > 1:
            print('epoch = %d : E = %12.10f'% (i+1, E * scale))

    if i == maxiter-1:
        print('Warning: maxiter exceeded. Sammon mapping may not have converged...')

    # Fiddle stress to match the original Sammon paper
    E = E * scale
    
    return [y,E]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import random_projection
from sklearn.manifold import Isomap
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.decomposition import KernelPCA
import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import manifold
from functools import partial
from collections import OrderedDict
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

#create new dataframes. they will contain principal components extracted from different dimensionality reduction techniques
#df_dimredu contains components extracted from outlierness scores
#df_dataredu contains components extracted from original data

df_dimredu = pd.DataFrame()
df_dataredu = pd.DataFrame()
df_binredu = pd.DataFrame()

def dim_redu_methods(data, scores_df, binary_scores_df):
  pca = PCA(n_components=2)
  out_pca = pd.DataFrame()
  out_pca_scores = pd.DataFrame()
  out_pca_bin = pd.DataFrame()
  try:
    pca.fit(data)
    out_pca = pd.DataFrame(pca.transform(data), index=data.index)
  except:
    print('Exception Raised --> PCA Config 1')

  try:
    pca.fit(scores_df)
    out_pca_scores = pd.DataFrame(pca.transform(scores_df), index=scores_df.index)
  except:
    print('Exception Raised --> PCA Config 2')

  try:
    pca.fit(binary_scores_df)
    out_pca_bin = pd.DataFrame(pca.transform(binary_scores_df), index=binary_scores_df.index)
  except:
    print('Exception Raised --> PCA Config 3')

  try:
    df_dataredu['pca1_data'] = out_pca[0]
    df_dataredu['pca2_data'] = out_pca[1]
    df_dimredu['pca1_scores'] = out_pca_scores[0]
    df_dimredu['pca2_scores'] = out_pca_scores[1]
    df_binredu['pca1_binary'] = out_pca_bin[0]
    df_binredu['pca2_binary'] = out_pca_bin[1]
  except:
    print('Exception Raised --> PCA Config loading')

  rsp = random_projection.GaussianRandomProjection(n_components=2)
  try:
    ran_proj = rsp.fit_transform(scores_df)
    df_dimredu['sub_proj1_scores'] = ran_proj[:,0]
    df_dimredu['sub_proj2_scores'] = ran_proj[:,1]
  except:
    print('Exception Raised --> Random Projection Config scores')

  try:  
    ran_proj_data = rsp.fit_transform(data)
    df_dataredu['sub_proj1_data'] = ran_proj_data[:,0]
    df_dataredu['sub_proj2_data'] = ran_proj_data[:,1]
  except:
    print('Exception Raised --> Random Projection Config data')

  try:
    ran_proj_bin = rsp.fit_transform(binary_scores_df)
    df_binredu['sub_proj1_binary'] = ran_proj_bin[:,0]
    df_binredu['sub_proj2_binary'] = ran_proj_bin[:,1]
  except:
    print('Exception Raised --> Random Projection Config binary')

  iso = Isomap(n_components=2)

  try:
    iso_proj = iso.fit_transform(scores_df)
    df_dimredu['iso_map_1_scores'] = iso_proj[:,0]
    df_dimredu['iso_map_2_scores'] = iso_proj[:,1]
  except:
    print('Exception Raised --> Iso Map Config scores')

  try:
    iso_proj_data = iso.fit_transform(data)
    df_dataredu['iso_map_1_data'] = iso_proj_data[:,0]
    df_dataredu['iso_map_2_data'] = iso_proj_data[:,1]
  except:
    print('Exception Raised --> Iso Map Config data')

  try:  
    iso_proj_bin = iso.fit_transform(binary_scores_df)
    df_binredu['iso_map_1_binary'] = iso_proj_bin[:,0]
    df_binredu['iso_map_2_binary'] = iso_proj_bin[:,1]
  except:
    print('Exception Raised --> Iso Map Config binary')

  tsne = TSNE(n_components=2)

  try:
    sne_proj = tsne.fit_transform(scores_df)
    df_dimredu['sne_1_scores'] = sne_proj[:,0]
    df_dimredu['sne_2_scores'] = sne_proj[:,1]
  except:
    print('Exception Raised --> TSNE Config scores')

  try:  
    sne_proj_data = tsne.fit_transform(data)
    df_dataredu['sne_1_data'] = sne_proj_data[:,0]
    df_dataredu['sne_2_data'] = sne_proj_data[:,1]
  except:
    print('Exception Raised --> TSNE Config data')

  try:
    sne_proj_bin = tsne.fit_transform(binary_scores_df)
    df_binredu['sne_1_binary'] = sne_proj_bin[:,0]
    df_binredu['sne_2_binary'] = sne_proj_bin[:,1]
  except:
    print('Exception Raised --> TSNE Config binary')

  clf = LinearDiscriminantAnalysis()

  try:
    clf.fit(scores_df, target)
    lda_proj = clf.transform(scores_df)
    df_dimredu['lda_1_scores_0'] = lda_proj[0:lda_proj.shape[0], 0]
    df_dimredu['lda_1_scores_1'] = lda_proj[0:lda_proj.shape[0], 1]
    df_dimredu['lda_1_scores_2'] = lda_proj[0:lda_proj.shape[0], 2]
    df_dimredu['lda_1_scores_3'] = lda_proj[0:lda_proj.shape[0], 3]
    df_dimredu['lda_1_scores_4'] = lda_proj[0:lda_proj.shape[0], 4]
    df_dimredu['lda_1_scores_5'] = lda_proj[0:lda_proj.shape[0], 5]
    df_dimredu['lda_1_scores_6'] = lda_proj[0:lda_proj.shape[0], 6]
    df_dimredu['lda_1_scores_7'] = lda_proj[0:lda_proj.shape[0], 7]
    df_dimredu['lda_1_scores_8'] = lda_proj[0:lda_proj.shape[0], 8]
    df_dimredu['lda_1_scores_9'] = lda_proj[0:lda_proj.shape[0], 9]
    df_dimredu['lda_1_scores_10'] = lda_proj[0:lda_proj.shape[0], 10]
  except:
    print('Exception Raised --> Linear Discriminant Analysis (classes exceeded)')

  try:
    clf.fit(data, target)
    lda_proj_data = clf.transform(data)
    df_dataredu['lda_1_data_0'] = lda_proj_data[0:lda_proj_data.shape[0], 0]
    df_dataredu['lda_1_data_1'] = lda_proj_data[0:lda_proj_data.shape[0], 1]
    df_dataredu['lda_1_data_2'] = lda_proj_data[0:lda_proj_data.shape[0], 2]
    df_dataredu['lda_1_data_3'] = lda_proj_data[0:lda_proj_data.shape[0], 3]
    df_dataredu['lda_1_data_4'] = lda_proj_data[0:lda_proj_data.shape[0], 4]
    df_dataredu['lda_1_data_5'] = lda_proj_data[0:lda_proj_data.shape[0], 5]
    df_dataredu['lda_1_data_6'] = lda_proj_data[0:lda_proj_data.shape[0], 6]
    df_dataredu['lda_1_data_7'] = lda_proj_data[0:lda_proj_data.shape[0], 7]
    df_dataredu['lda_1_data_8'] = lda_proj_data[0:lda_proj_data.shape[0], 8]
    df_dataredu['lda_1_data_9'] = lda_proj_data[0:lda_proj_data.shape[0], 9]
    df_dataredu['lda_1_data_10'] = lda_proj_data[0:lda_proj_data.shape[0], 10]
  except:
    print('Exception Raised --> Linear Discriminant Analysis Config (classes exceeded)')

  try:
    clf.fit(binary_scores_df, target)
    lda_proj_bin = clf.transform(binary_scores_df) 
    df_binredu['lda_1_binary_0'] = lda_proj_bin[0:lda_proj_bin.shape[0], 0]
    df_binredu['lda_1_binary_1'] = lda_proj_bin[0:lda_proj_bin.shape[0], 1]
    df_binredu['lda_1_binary_2'] = lda_proj_bin[0:lda_proj_bin.shape[0], 2]
    df_binredu['lda_1_binary_3'] = lda_proj_bin[0:lda_proj_bin.shape[0], 3]
    df_binredu['lda_1_binary_4'] = lda_proj_bin[0:lda_proj_bin.shape[0], 4]
    df_binredu['lda_1_binary_5'] = lda_proj_bin[0:lda_proj_bin.shape[0], 5]
    df_binredu['lda_1_binary_6'] = lda_proj_bin[0:lda_proj_bin.shape[0], 6]
    df_binredu['lda_1_binary_7'] = lda_proj_bin[0:lda_proj_bin.shape[0], 7]
    df_binredu['lda_1_binary_8'] = lda_proj_bin[0:lda_proj_bin.shape[0], 8]
    df_binredu['lda_1_binary_9'] = lda_proj_bin[0:lda_proj_bin.shape[0], 9]
    df_binredu['lda_1_binary_10'] = lda_proj_bin[0:lda_proj_bin.shape[0], 10]
  except:
    print('Exception Raised --> Linear Discriminant Analysis Config (classes exceeded)')

  kpca = KernelPCA(n_components=2, kernel='rbf', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['rbf_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['rbf_pca_2_scores'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 1')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['rbf_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['rbf_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 1')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['rbf_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['rbf_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 1')


  kpca = KernelPCA(n_components=2, kernel='cosine', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['cosine_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['cosine_pca_2_scores'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 2')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['cosine_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['cosine_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 2')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['cosine_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['cosine_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 2')

  kpca = KernelPCA(n_components=2, kernel='sigmoid', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['sigmoid_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['sigmoid_pca_2_scores'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 3')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['sigmoid_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['sigmoid_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 3')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['sigmoid_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['sigmoid_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 3')

  kpca = KernelPCA(n_components=2, kernel='poly', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['poly_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['poly_pca_2_scores_'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 4')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['poly_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['poly_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 4')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['poly_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['poly_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 4')

  #convert two scores in numeric, to perform sammon function
  if 'pynomaly_loop_original' in scores_df.columns:
    scores_df["pynomaly_loop_original"] = pd.to_numeric(scores_df["pynomaly_loop_original"])
  #scores_df["dbscan_scores_original"] = pd.to_numeric(scores_df["dbscan_scores_original"])

  try:
    # By default, sammon returns a 2-dim array and the error E
    [y, E] = sammon(data, maxiter=20, n=2)
    [y2, E2] = sammon(scores_df, maxiter=20, n=2)

    df_dataredu['sammon_1_data'] = y[:,0]
    df_dataredu['sammon_2_data'] = y[:,1]
    df_dimredu['sammon_1_scores'] = y2[:,0]
    df_dimredu['sammon_2_scores'] = y2[:,1]
  except:
    print('Exception Raised --> Sammon Mapping Config')

  try:
    Axes3D
    n_neighbors = 100
    n_components = 2
    # Set-up manifold methods
    LLE = partial(manifold.LocallyLinearEmbedding, n_neighbors=n_neighbors, n_components=n_components, eigen_solver="auto")
    methods = OrderedDict()
    methods["LLE"] = LLE(method="standard")
    methods["Hessian LLE"] = LLE(method="hessian", eigen_solver='dense')
    methods["Modified LLE"] = LLE(method="modified", eigen_solver='dense')
    methods["MDS"] = manifold.MDS(n_components, max_iter=100, n_init=1)
    methods["SE"] = manifold.SpectralEmbedding(n_components=n_components, n_neighbors=n_neighbors)
  except:
    print('Exception Raised -> MainFold Methods Inizialization')
  try:
    for i, (label, method) in enumerate(methods.items()):
      Y = method.fit_transform(scores_df)
      Y2 = method.fit_transform(data)
      str1 = label+"_1"
      str2 = label+"_2"  
      
      if(label!='Modified LLE'):
        Y3 = method.fit_transform(binary_scores_df)
        df_binredu[str1+"_binary"] = Y3[:,0]
        df_binredu[str2+"_binary"] = Y3[:,1]      
      
      df_dimredu[str1+"_scores"] = Y[:,0]
      df_dimredu[str2+"_scores"] = Y[:,1]
      df_dataredu[str1+"_data"] = Y2[:,0]
      df_dataredu[str2+"_data"] = Y2[:,1]
      print("%s Completed" % (label))
  except:
    print('Exception Raised --> MainFold Methods Calculus')

  try:
    # Fixed dimensions
    input_dim = data.shape[1]
    encoding_dim = 2
    # Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
    input_layer = Input(shape=(input_dim, ))
    encoder_layer_1 = Dense(18, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(9, activation="tanh")(encoder_layer_1)
    encoder_layer_3 = Dense(7, activation="tanh")(encoder_layer_2)
    encoder_layer_4 = Dense(5, activation="relu")(encoder_layer_3)
    encoder_layer_5 = Dense(3, activation="relu")(encoder_layer_4)
    encoder_layer_6 = Dense(encoding_dim, activation="sigmoid")(encoder_layer_5)

    # Crear encoder model
    encoder = Model(inputs=input_layer, outputs=encoder_layer_6)
    # Use the model to predict the factors which sum up the information of interest rates.
    encoded_data = pd.DataFrame(encoder.predict(data))
    encoded_data.columns = ['factor_1', 'factor_2']

    # Fixed dimensions
    input_dim = scores_df.shape[1]
    encoding_dim = 2
    # Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
    input_layer = Input(shape=(input_dim, ))
    encoder_layer_1 = Dense(18, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(9, activation="relu")(encoder_layer_1)
    encoder_layer_3 = Dense(7, activation="relu")(encoder_layer_2)
    encoder_layer_4 = Dense(5, activation="relu")(encoder_layer_3)
    encoder_layer_5 = Dense(3, activation="relu")(encoder_layer_4)
    encoder_layer_6 = Dense(encoding_dim, activation="sigmoid")(encoder_layer_5)

    # Crear encoder model
    encoder = Model(inputs=input_layer, outputs=encoder_layer_6)
    # Use the model to predict the factors which sum up the information of interest rates.
    encoded_scores = pd.DataFrame(encoder.predict(scores_df))
    encoded_scores.columns = ['factor_1', 'factor_2']

    # Fixed dimensions
    input_dim = binary_scores_df.shape[1]
    encoding_dim = 2
    # Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
    input_layer = Input(shape=(input_dim, ))
    encoder_layer_1 = Dense(18, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(9, activation="relu")(encoder_layer_1)
    encoder_layer_3 = Dense(7, activation="relu")(encoder_layer_2)
    encoder_layer_4 = Dense(5, activation="relu")(encoder_layer_3)
    encoder_layer_5 = Dense(3, activation="relu")(encoder_layer_4)
    encoder_layer_6 = Dense(encoding_dim, activation="sigmoid")(encoder_layer_5)

    # Crear encoder model
    encoder = Model(inputs=input_layer, outputs=encoder_layer_6)
    # Use the model to predict the factors which sum up the information of interest rates.
    encoded_bin = pd.DataFrame(encoder.predict(binary_scores_df))
    encoded_bin.columns = ['factor_1', 'factor_2']

    df_dataredu['autoencoder_1_data'] = encoded_data['factor_1']
    df_dataredu['autoencoder_2_data'] = encoded_data['factor_2']
    df_dimredu['autoencoder_1_scores'] = encoded_scores['factor_1']
    df_dimredu['autoencoder_2_scores'] = encoded_scores['factor_2']
    df_binredu['autoencoder_1_binary'] = encoded_bin['factor_1']
    df_binredu['autoencoder_2_binary'] = encoded_bin['factor_2']
  except:
    print('Exception Raised --> AutoEncoder Config')

#APPLY FEATURE EXTRACTION PIPELINE ON ORIGINAL DATA

In [ ]:
def extract_outlierness_features(data, pynomaly=True, sklearn=True, pyod=True, network=True):
  if(pynomaly==True):
    pynomaly_methods(data, scores_df, binary_scores_df)
  if(sklearn==True):
    sklearn_methods(data, scores_df, binary_scores_df)
  if(pyod==True):
    pyod_methods(data, scores_df, binary_scores_df)
  if(network==True):
    network_methods(data)

In [ ]:
import time

start_time = time.time()

print('Outlierness Scores Extraction: \n')
extract_outlierness_features(data, pynomaly=True,sklearn=True,pyod=True,network=False)

#normalize outlierness scores
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scores_df = pd.DataFrame(scaler.fit_transform(scores_df), columns=scores_df.columns)

print('\nNull Values in Outlierness Scores DataSets:')
print('Scores_df: ')
scores_df.loc[:, scores_df.isna().any()]
scores_df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
print('--- --- ---')
print('Binary_Scores_df: ')
binary_scores_df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
binary_scores_df.loc[:, binary_scores_df.isna().any()]
print('--- --- ---')

#save outlierness numeric and binary scores to .csv
scores_df.to_csv(directory + "outlierness_scores.csv")
binary_scores_df.to_csv(directory + "binary_scores.csv")

#apply dimensionality reduction methods on original data, numeric and binary outlierness scores
print('\n--- --- --- --- ---\nDimensionality Reduction Features Extraction: ')
dim_redu_methods(data, scores_df, binary_scores_df)

print('\nNull Values in Dimensionality Reduction DataSets:')
df_dimredu.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
print(df_dimredu.isnull().sum())
print('--- --- ---')
df_dataredu.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
print(df_dataredu.isnull().sum())
print('--- --- ---')
df_binredu.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
print(df_binredu.isnull().sum())
print('--- --- ---')

#normalize calculated principal components
scaler = RobustScaler()
df_dimredu = pd.DataFrame(scaler.fit_transform(df_dimredu), columns=df_dimredu.columns)
df_dataredu = pd.DataFrame(scaler.fit_transform(df_dataredu), columns=df_dataredu.columns)
df_binredu = pd.DataFrame(scaler.fit_transform(df_binredu), columns=df_binredu.columns)

end = time.time()
#save first feature extraction phase time ellapsed
print("Support Features Extraction Time: %.8s seconds" % (end - start_time))

#save dimensionality reduction (applied on original data, binary and numeric outlierness scores) to .csv
df_dimredu.to_csv(directory + "dim_redu_scores.csv")
df_dataredu.to_csv(directory + "dim_redu_original_data.csv")
df_binredu.to_csv(directory + "dim_redu_binary_scores.csv")

#OUTLIERNESS SCORES EXTRACTION ON PRINCIPAL COMPONENTS

In [ ]:
data = pd.read_csv(directory+'dim_redu_original_data.csv', delimiter=',') #load data
data.drop('Unnamed: 0', inplace=True, axis=1)

original = pd.read_csv(directory+dataname, delimiter=',') #load data
original.drop('Unnamed: 0', inplace=True, axis=1)
target = original['class']

##pynomaly

In [ ]:
scores_df = pd.DataFrame()
binary_scores_df = pd.DataFrame()

In [ ]:
def pynomaly_methods(data, scores_df, binary_scores_df):
  try:
    m = loop.LocalOutlierProbability(data, progress_bar=True).fit()
    scores = m.local_outlier_probabilities

    scores_df = pd.DataFrame(scores, columns = ['pynomaly_loop_dim_redu'])
    binary_scores_df = pd.DataFrame(scores, columns = ['pynomaly_loop_dim_redu_binary'])
    binary_scores_df['pynomaly_loop_dim_redu_binary'] = np.where(binary_scores_df['pynomaly_loop_dim_redu_binary']>=0.5, 1, 0)
  except:
    print('Exception Raised -> LOP Config')

  try:
    db = DBSCAN(eps=0.1, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps01_sam10_nei10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam10_nei10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam10_nei10_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam10_nei10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 1')

  try:
    db = DBSCAN(eps=0.1, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps01_sam20_nei10_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam20_nei10_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam20_nei10_e2'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam20_nei10_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 2')

  try:
    db = DBSCAN(eps=0.1, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, n_neighbors=20, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps01_sam20_nei20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam20_nei20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam20_nei20_e2'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam20_nei20_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 3')

  try:
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps01_sam50_nei10_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei10_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei10_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei10_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 4')

  try:
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=20, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps01_sam50_nei20_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei20_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei20_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei20_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 5')

  try:
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=50, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps01_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei50_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps01_sam50_nei50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 6')

  try:
    db = DBSCAN(eps=0.3, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps03_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam10_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 7')

  try:
    db = DBSCAN(eps=0.3, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps03_sam20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam20_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam20_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam20_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 8')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2,  cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps03_sam50_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e2'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 9')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3,cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 10')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps03_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 11')

  try:
    db = DBSCAN(eps=0.5, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1,cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps05_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam10_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 12')

  try:
    db = DBSCAN(eps=0.5, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps05_sam20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam20_e2'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam20_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 13')

  try:
    db = DBSCAN(eps=0.5, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps05_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam50_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps05_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 14')

  try:
    db = DBSCAN(eps=0.9, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps09_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam10_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam10_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 15')

  try:
    db = DBSCAN(eps=0.9, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps09_sam20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam20_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam20_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam20_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 16')

  try:
    db = DBSCAN(eps=0.9, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps09_sam50_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_e1'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_e1'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 17')

  try:
    db = DBSCAN(eps=0.9, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps09_sam50_nei20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_nei20_e2'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_nei20_e2'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_nei20_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 18')

  try:
    db = DBSCAN(eps=0.9, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['dbscan_scores_dim_redu_eps09_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_nei50_e3'] = np.where(binary_scores_df['dbscan_scores_dim_redu_binary_eps09_sam50_nei50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised -> DBSCAN Config 19')

##sklearn algorithms

In [ ]:
from sklearn.svm import OneClassSVM

def sklearn_methods(data, scores_df, binary_scores_df):
  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.001)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_dim_redu_cont0.001'] = dec_func
    scores_df['elliptic_mahalanobis_dim_redu_cont0.001'] = maha
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.001'] = pred
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.001'] = np.where(binary_scores_df['elliptic_env_dim_redu_binary_cont0.001']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 1')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.01)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_dim_redu_cont0.01'] = dec_func
    scores_df['elliptic_mahalanobis_dim_redu_cont0.01'] = maha
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.01'] = pred
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.01'] = np.where(binary_scores_df['elliptic_env_dim_redu_binary_cont0.01']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 2')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.1)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_dim_redu_cont0.1'] = dec_func
    scores_df['elliptic_mahalanobis_dim_redu_cont0.1'] = maha
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.1'] = pred
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.1'] = np.where(binary_scores_df['elliptic_env_dim_redu_binary_cont0.1']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 3')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.2)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_dim_redu_cont0.2'] = dec_func
    scores_df['elliptic_mahalanobis_dim_redu_cont0.2'] = maha
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.2'] = pred
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.2'] = np.where(binary_scores_df['elliptic_env_dim_redu_binary_cont0.2']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 4')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.5)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['elliptic_decfunc_dim_redu_cont0.5'] = dec_func
    scores_df['elliptic_mahalanobis_dim_redu_cont0.5'] = maha
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.5'] = pred
    binary_scores_df['elliptic_env_dim_redu_binary_cont0.5'] = np.where(binary_scores_df['elliptic_env_dim_redu_binary_cont0.5']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 5')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.001, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_dim_redu_cont0.001'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.001'] = pred
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.001'] = np.where(binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.001']==-1, 1,0)
  except:
    print('Exception Raised --> Isolation Forest Config 1')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.01, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_dim_redu_cont0.01'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.01'] = pred
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.01'] = np.where(binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.01']==-1, 1,0)
  except:
    print('Exception Raised --> Isolation Forest Config 2')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.1, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_dim_redu_cont0.1'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.1'] = pred
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.1'] = np.where(binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.1']==-1, 1,0)
  except:
    print('Exception Raised --> Isolation Forest Config 3')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.2, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_dim_redu_cont0.2'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.2'] = pred
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.2'] = np.where(binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.2']==-1, 1,0)
  except:
    print('Exception Raised --> Isolation Forest Config 4')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.5, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['iso_forest_paper_score_dim_redu_cont0.5'] = original_paper_score
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.5'] = pred
    binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.5'] = np.where(binary_scores_df['iso_forest_paper_score_dim_redu_binary_cont0.5']==-1, 1,0)
  except:
    print('Exception Raised --> Isolation Forest Config 5')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=10, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['lof_score_dim_redu_mink_nei10'] = lof_score
    binary_scores_df['lof_score_dim_redu_binary_mink_nei10'] = pred
    binary_scores_df['lof_score_dim_redu_binary_mink_nei10'] = np.where(binary_scores_df['lof_score_dim_redu_binary_mink_nei10']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 1')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=20, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['lof_score_dim_redu_mink_nei20'] = lof_score
    binary_scores_df['lof_score_dim_redu_binary_mink_nei20'] = pred
    binary_scores_df['lof_score_dim_redu_binary_mink_nei20'] = np.where(binary_scores_df['lof_score_dim_redu_binary_mink_nei20']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 2')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=50, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['lof_score_dim_redu_mink_nei50'] = lof_score
    binary_scores_df['lof_score_dim_redu_binary_mink_nei50'] = pred
    binary_scores_df['lof_score_dim_redu_binary_mink_nei50'] = np.where(binary_scores_df['lof_score_dim_redu_binary_mink_nei50']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 3')

  # try:
  #   ee = OneClassSVM(nu=0.001, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_dim_redu_linear_nu0.001'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.001'] = pred
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.001'] = np.where(binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.001']==-1, 1,0)
  # except:
  #   print('Exception Raised --> OneClass SVM Config 1')

  # try:
  #   ee = OneClassSVM(nu=0.01, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_dim_redu_linear_nu0.01'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.01'] = pred
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.01'] = np.where(binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.01']==-1, 1,0)
  # except:
  #   print('Exception Raised --> OneClass SVM Config 2')

  # try:
  #   ee = OneClassSVM(nu=0.1, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_dim_redu_linear_nu0.1'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.1'] = pred
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.1'] = np.where(binary_scores_df['oneclass_SVM_dim_redu_binary_linear_nu0.1']==-1, 1,0)
  # except:
  #   print('Exception Raised --> OneClass SVM Config 3')

  # try:
  #   ee = OneClassSVM(nu=0.001)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_dim_redu_rbf_nu0.001'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.001'] = pred
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.001'] = np.where(binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.001']==-1, 1,0)
  # except:
  #   print('Exception Raised --> OneClass SVM Config 4')

  # try:
  #   ee = OneClassSVM(nu=0.01)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_dim_redu_rbf_nu0.01'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.01'] = pred
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.01'] = np.where(binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.01']==-1, 1,0)
  # except:
  #   print('Exception Raised --> OneClass SVM Config 5')

  # try:
  #   ee = OneClassSVM(nu=0.1)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['oneclass_SVM_dim_redu_rbf_nu0.1'] = ocsvm_score
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.1'] = pred
  #   binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.1'] = np.where(binary_scores_df['oneclass_SVM_dim_redu_binary_rbf_nu0.1']==-1, 1,0)
  # except:
  #   print('Exception Raised --> OneClass SVM Config 6')

##pyod

In [ ]:
from pyod.models.copod import COPOD
from pyod.models.suod import SUOD
from pyod.models.loda import LODA
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.cblof import CBLOF
from pyod.models.cof import COF
from pyod.models.ocsvm import OCSVM
from pyod.models.mcd import MCD
from pyod.models.pca import PCA
from pyod.models.ecod import ECOD
from pyod.models.sos import SOS

def pyod_methods(data, scores_df, binary_scores_df):
  try:
    clf = COPOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_dim_redu_cont0.001'] = pyod_copod
    binary_scores_df['pyod_copod_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> COPOD Config 1')

  try:
    clf = COPOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_dim_redu_cont0.01'] = pyod_copod
    binary_scores_df['pyod_copod_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> COPOD Config 2')

  try:
    clf = COPOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_dim_redu_cont0.1'] = pyod_copod
    binary_scores_df['pyod_copod_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> COPOD Config 3')

  try:
    clf = COPOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_dim_redu_cont0.2'] = pyod_copod
    binary_scores_df['pyod_copod_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> COPOD Config 4')

  try:
    clf = COPOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['pyod_copod_dim_redu_cont0.5'] = pyod_copod
    binary_scores_df['pyod_copod_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> COPOD Config 5')

  try:
    clf = ECOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_dim_redu_cont0.001'] = pyod_ecod
    binary_scores_df['pyod_ecod_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> ECOD Config 1')

  try:
    clf = ECOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_dim_redu_cont0.01'] = pyod_ecod
    binary_scores_df['pyod_ecod_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> ECOD Config 2')

  try:
    clf = ECOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_dim_redu_cont0.1'] = pyod_ecod
    binary_scores_df['pyod_ecod_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> ECOD Config 3')

  try:
    clf = ECOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_dim_redu_cont0.2'] = pyod_ecod
    binary_scores_df['pyod_ecod_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> ECOD Config 4')

  try:
    clf = ECOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['pyod_ecod_dim_redu_cont0.5'] = pyod_ecod
    binary_scores_df['pyod_ecod_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> ECOD Config 5')

  try:
    clf = SOS(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_dim_redu_cont0.001'] = pyod_sos
    binary_scores_df['pyod_sos_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> SOS Config 1')

  try:
    clf = SOS(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_dim_redu_cont0.01'] = pyod_sos
    binary_scores_df['pyod_sos_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> SOS Config 2')

  try:
    clf = SOS(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_dim_redu_cont0.1'] = pyod_sos
    binary_scores_df['pyod_sos_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> SOS Config 3')

  try:
    clf = SOS(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_dim_redu_cont0.2'] = pyod_sos
    binary_scores_df['pyod_sos_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> SOS Config 4')

  try:
    clf = SOS(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['pyod_sos_dim_redu_cont0.5'] = pyod_sos
    binary_scores_df['pyod_sos_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> SOS Config 5')

  try:
    clf = PCA(n_components=2, contamination=0.001, whiten=False)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.001_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.001_whitFalse'] = pred
  except:
    print('Exception Raised --> PCA Config 1')

  try:
    clf = PCA(n_components=2, contamination=0.01, whiten=False)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.01_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.01_whitFalse'] = pred
  except:
    print('Exception Raised --> PCA Config 2')

  try:
    clf = PCA(n_components=2, contamination=0.1, whiten=False)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.1_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.1_whitFalse'] = pred
  except:
    print('Exception Raised --> PCA Config 3')

  try:
    clf = PCA(n_components=2, contamination=0.2, whiten=False)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.2_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.2_whitFalse'] = pred
  except:
    print('Exception Raised --> PCA Config 4')

  try:
    clf = PCA(n_components=2, contamination=0.5, whiten=False)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.5_whitFalse'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.5_whitFalse'] = pred
  except:
    print('Exception Raised --> PCA Config 5')

  try:
    clf = PCA(n_components=2, contamination=0.001, whiten=True)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.001_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.001_whitTrue'] = pred
  except:
    print('Exception Raised --> PCA Config 6')

  try:
    clf = PCA(n_components=2, contamination=0.01, whiten=True)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.01_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.01_whitTrue'] = pred
  except:
    print('Exception Raised --> PCA Config 7')

  try:
    clf = PCA(n_components=2, contamination=0.1, whiten=True)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.1_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.1_whitTrue'] = pred
  except:
    print('Exception Raised --> PCA Config 8')

  try:
    clf = PCA(n_components=2, contamination=0.2, whiten=True)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.2_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.2_whitTrue'] = pred
  except:
    print('Exception Raised --> PCA Config 9')

  try:
    clf = PCA(n_components=2, contamination=0.5, whiten=True)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['pyod_pca_dim_redu_cont0.5_whitTrue'] = pyod_pca
    binary_scores_df['pyod_pca_dim_redu_binary_cont0.5_whitTrue'] = pred
  except:
    print('Exception Raised --> PCA Config 10')

  try:
    clf = MCD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_dim_redu_cont0.001'] = pyod_mcd
    binary_scores_df['pyod_mcd_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> MCD Config 1')

  try:
    clf = MCD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_dim_redu_cont0.01'] = pyod_mcd
    binary_scores_df['pyod_mcd_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> MCD Config 2')

  try:
    clf = MCD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_dim_redu_cont0.1'] = pyod_mcd
    binary_scores_df['pyod_mcd_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> MCD Config 3')

  try:
    clf = MCD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_dim_redu_cont0.2'] = pyod_mcd
    binary_scores_df['pyod_mcd_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> MCD Config 4')

  try:
    clf = MCD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['pyod_mcd_dim_redu_cont0.5'] = pyod_mcd
    binary_scores_df['pyod_mcd_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> MCD Config 5')

  try:
    clf = OCSVM(contamination=0.001, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.001_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.001'] = pred
  except:
    print('Exception Raised --> OCSVM Config 1')

  try:
    clf = OCSVM(contamination=0.01, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.01_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.01_nu0.001'] = pred
  except:
    print('Exception Raised --> OCSVM Config 2')

  try:
    clf = OCSVM(contamination=0.1, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.1_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.1_nu0.001'] = pred
  except:
    print('Exception Raised --> OCSVM Config 3')

  try:
    clf = OCSVM(contamination=0.2, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.2_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.2_nu0.001'] = pred
  except:
    print('Exception Raised --> OCSVM Config 4')

  try:
    clf = OCSVM(contamination=0.5, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.5_nu0.001'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.5_nu0.001'] = pred
  except:
    print('Exception Raised --> OCSVM Config 5')

  try:
    clf = OCSVM(contamination=0.001, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.001_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.1'] = pred
  except:
    print('Exception Raised --> OCSVM Config 6')

  try:
    clf = OCSVM(contamination=0.001, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.001_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.3'] = pred
  except:
    print('Exception Raised --> OCSVM Config 7')

  try:  
    clf = OCSVM(contamination=0.001, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.001_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.5'] = pred
  except:
    print('Exception Raised --> OCSVM Config 8')

  try:
    clf = OCSVM(contamination=0.01, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.01_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.1'] = pred
  except:
    print('Exception Raised --> OCSVM Config 9')

  try:
    clf = OCSVM(contamination=0.01, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.01_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.3'] = pred
  except:
    print('Exception Raised --> OCSVM Config 10')

  try:
    clf = OCSVM(contamination=0.01, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.01_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.001_nu0.5'] = pred
  except:
    print('Exception Raised --> OCSVM Config 11')

  try:
    clf = OCSVM(contamination=0.1, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.1_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.1_nu0.1'] = pred
  except:
    print('Exception Raised --> OCSVM Config 12')

  try:
    clf = OCSVM(contamination=0.1, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.1_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.1_nu0.3'] = pred
  except:
    print('Exception Raised --> OCSVM Config 13')

  try:
    clf = OCSVM(contamination=0.1, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.1_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.1_nu0.5'] = pred
  except:
    print('Exception Raised --> OCSVM Config 14')

  try:
    clf = OCSVM(contamination=0.2, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.2_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.2_nu0.1'] = pred
  except:
    print('Exception Raised --> OCSVM Config 15')

  try:
    clf = OCSVM(contamination=0.2, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.2_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.2_nu0.3'] = pred
  except:
    print('Exception Raised --> OCSVM Config 16')

  try:
    clf = OCSVM(contamination=0.2, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.2_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.2_nu0.5'] = pred
  except:
    print('Exception Raised --> OCSVM Config 17')

  try:
    clf = OCSVM(contamination=0.5, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.5_nu0.1'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.5_nu0.1'] = pred
  except:
    print('Exception Raised --> OCSVM Config 18')

  try:
    clf = OCSVM(contamination=0.5, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.5_nu0.3'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.5_nu0.3'] = pred
  except:
    print('Exception Raised --> OCSVM Config 19')

  try:
    clf = OCSVM(contamination=0.5, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['pyod_ocsvm_dim_redu_cont0.5_nu0.5'] = pyod_ocsvm
    binary_scores_df['pyod_ocsvm_dim_redu_binary_cont0.5_nu0.5'] = pred
  except:
    print('Exception Raised --> OCSVM Config 20')

  try:
    clf = COF(contamination=0.001, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.001_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.001_nei10'] = pred
  except:
    print('Exception Raised --> COF Config 1')

  try:
    clf = COF(contamination=0.01, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.01_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.01_nei10'] = pred
  except:
    print('Exception Raised --> COF Config 2')

  try:
    clf = COF(contamination=0.1, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.1_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.1_nei10'] = pred
  except:
    print('Exception Raised --> COF Config 3')

  try:
    clf = COF(contamination=0.2, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.2_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.2_nei10'] = pred
  except:
    print('Exception Raised --> COF Config 4')

  try:
    clf = COF(contamination=0.5, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.5_nei10'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.5_nei10'] = pred
  except:
    print('Exception Raised --> COF Config 5')

  try:
    clf = COF(contamination=0.001, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.001_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.001_nei20'] = pred
  except:
    print('Exception Raised --> COF Config 6')

  try:
    clf = COF(contamination=0.01, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.01_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.01_nei20'] = pred
  except:
    print('Exception Raised --> COF Config 7')

  try:
    clf = COF(contamination=0.1, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.1_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.1_nei20'] = pred
  except:
    print('Exception Raised --> COF Config 8')

  try:
    clf = COF(contamination=0.2, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.2_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.2_nei20'] = pred
  except:
    print('Exception Raised --> COF Config 9')

  try:
    clf = COF(contamination=0.5, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.5_nei20'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.5_nei20'] = pred
  except:
    print('Exception Raised --> COF Config 10')

  try:
    clf = COF(contamination=0.001, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.001_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.001_nei50'] = pred
  except:
    print('Exception Raised --> COF Config 11')

  try:
    clf = COF(contamination=0.01, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.01_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.01_nei50'] = pred
  except:
    print('Exception Raised --> COF Config 12')

  try:
    clf = COF(contamination=0.1, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.1_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.1_nei50'] = pred
  except:
    print('Exception Raised --> COF Config 13')

  try:
    clf = COF(contamination=0.2, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.2_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.2_nei50'] = pred
  except:
    print('Exception Raised --> COF Config 14')

  try:
    clf = COF(contamination=0.5, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['pyod_cof_dim_redu_cont0.5_nei50'] = pyod_cof
    binary_scores_df['pyod_cof_dim_redu_binary_cont0.5_nei50'] = pred
  except:
    print('Exception Raised --> COF Config 15')

  try:
    clf = CBLOF(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_dim_redu_cont0.001'] = pyod_cblof
    binary_scores_df['pyod_cblof_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> CBLOF Config 1')

  try:
    clf = CBLOF(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_dim_redu_cont0.01'] = pyod_cblof
    binary_scores_df['pyod_cblof_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> CBLOF Config 2')

  try:
    clf = CBLOF(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_dim_redu_cont0.1'] = pyod_cblof
    binary_scores_df['pyod_cblof_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> CBLOF Config 3')

  try:
    clf = CBLOF(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_dim_redu_cont0.2'] = pyod_cblof
    binary_scores_df['pyod_cblof_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> CBLOF Config 4')

  try:
    clf = CBLOF(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['pyod_cblof_dim_redu_cont0.5'] = pyod_cblof
    binary_scores_df['pyod_cblof_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> CBLOF Config 5')

  try:
    clf = HBOS(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_dim_redu_cont0.001'] = pyod_hbos
    binary_scores_df['pyod_hbos_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> HBOS Config 1')

  try:
    clf = HBOS(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_dim_redu_cont0.01'] = pyod_hbos
    binary_scores_df['pyod_hbos_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> HBOS Config 2')

  try:
    clf = HBOS(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_dim_redu_cont0.1'] = pyod_hbos
    binary_scores_df['pyod_hbos_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> HBOS Config 3')

  try:
    clf = HBOS(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_dim_redu_cont0.2'] = pyod_hbos
    binary_scores_df['pyod_hbos_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> HBOS Config 4')

  try:
    clf = HBOS(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['pyod_hbos_dim_redu_cont0.5'] = pyod_hbos
    binary_scores_df['pyod_hbos_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> HBOS Config 5')

  try:
    clf = KNN(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_dim_redu_cont0.001'] = pyod_knn
    binary_scores_df['pyod_knn_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> KNN Config 1')

  try:
    clf = KNN(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_dim_redu_cont0.01'] = pyod_knn
    binary_scores_df['pyod_knn_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> KNN Config 2')

  try:
    clf = KNN(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_dim_redu_cont0.1'] = pyod_knn
    binary_scores_df['pyod_knn_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> KNN Config 3')

  try:
    clf = KNN(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_dim_redu_cont0.2'] = pyod_knn
    binary_scores_df['pyod_knn_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> KNN Config 4')

  try:
    clf = KNN(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['pyod_knn_dim_redu_cont0.5'] = pyod_knn
    binary_scores_df['pyod_knn_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> KNN Config 5')

  try:
    clf = FeatureBagging(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_dim_redu_cont0.001'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> Feature Bagging Config 1')

  try:
    clf = FeatureBagging(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_dim_redu_cont0.01'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> Feature Bagging Config 2')

  try:
    clf = FeatureBagging(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_dim_redu_cont0.1'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> Feature Bagging Config 3')

  try:
    clf = FeatureBagging(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_dim_redu_cont0.2'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> Feature Bagging Config 4')

  try:
    clf = FeatureBagging(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['pyod_featbagg_dim_redu_cont0.5'] = pyod_featbagg
    binary_scores_df['pyod_featbagg_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> Feature Bagging Config 5')

  try:
    clf = LODA(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_dim_redu_cont0.001'] = pyod_loda
    binary_scores_df['pyod_loda_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> LODA Config 1')

  try:
    clf = LODA(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_dim_redu_cont0.01'] = pyod_loda
    binary_scores_df['pyod_loda_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> LODA Config 2')

  try:
    clf = LODA(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_dim_redu_cont0.1'] = pyod_loda
    binary_scores_df['pyod_loda_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> LODA Config 3')

  try:
    clf = LODA(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_dim_redu_cont0.2'] = pyod_loda
    binary_scores_df['pyod_loda_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> LODA Config 4')

  try:
    clf = LODA(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['pyod_loda_dim_redu_cont0.5'] = pyod_loda
    binary_scores_df['pyod_loda_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> LODA Config 5')

  try:
    clf = SUOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_dim_redu_cont0.001'] = pyod_suod
    binary_scores_df['pyod_suod_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> SUOD Config 1')

  try:
    clf = SUOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_dim_redu_cont0.01'] = pyod_suod
    binary_scores_df['pyod_suod_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> SUOD Config 2')

  try:
    clf = SUOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_dim_redu_cont0.1'] = pyod_suod
    binary_scores_df['pyod_suod_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> SUOD Config 3')

  try:
    clf = SUOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_dim_redu_cont0.2'] = pyod_suod
    binary_scores_df['pyod_suod_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> SUOD Config 4')

  try:
    clf = SUOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['pyod_suod_dim_redu_cont0.5'] = pyod_suod
    binary_scores_df['pyod_suod_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> SUOD Config 5')

network methods

In [ ]:
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.deep_svdd import DeepSVDD
from pyod.models.mo_gaal import MO_GAAL
import pyod.models.deep_svdd
from pyod.models.so_gaal import SO_GAAL
from pyod.models.vae import VAE

def network_methods(data, scores_df, binary_scores_df):
  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_dim_redu_cont0.001'] = pyod_encoder
    binary_scores_df['pyod_encoder_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> AutoEncoder Config 1')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_dim_redu_cont0.01'] = pyod_encoder
    binary_scores_df['pyod_encoder_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> AutoEncoder Config 2')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_dim_redu_cont0.1'] = pyod_encoder
    binary_scores_df['pyod_encoder_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> AutoEncoder Config 3')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_dim_redu_cont0.2'] = pyod_encoder
    binary_scores_df['pyod_encoder_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> AutoEncoder Config 4')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['pyod_encoder_dim_redu_cont0.5'] = pyod_encoder
    binary_scores_df['pyod_encoder_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> AutoEncoder Config 5')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_dim_redu_cont0.001'] = pyod_vae
    binary_scores_df['pyod_vae_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> VAE Config 1')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_dim_redu_cont0.01'] = pyod_vae
    binary_scores_df['pyod_vae_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> VAE Config 2')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_dim_redu_cont0.1'] = pyod_vae
    binary_scores_df['pyod_vae_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> VAE Config 3')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_dim_redu_cont0.2'] = pyod_vae
    binary_scores_df['pyod_vae_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> VAE Config 4')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['pyod_vae_dim_redu_cont0.5'] = pyod_vae
    binary_scores_df['pyod_vae_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> VAE Config 5')

  try:
    clf = SO_GAAL(contamination=0.001, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.001_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.001_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 1')

  try:
    clf = SO_GAAL(contamination=0.01, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.01_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.01_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 2')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.1_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.1_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 3')

  try:
    clf = SO_GAAL(contamination=0.2, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.2_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.2_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 4')

  try:
    clf = SO_GAAL(contamination=0.5, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.5_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.5_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 5')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.1_lrd0.01_lrg0.01_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.1_lrd0.01_lrg0.01_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 6')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.3 , lr_g=0.3 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.1_lrd0.3_lrg0.3_mom0.01'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.1_lrd0.3_lrg0.3_mom0.01'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 7')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.1_lrd0.01_lrg0.01_mom0.1'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.1_lrd0.01_lrg0.01_mom0.1'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 8')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['pyod_sogaal_dim_redu_cont0.1_lrd0.01_lrg0.01_mom0.3'] = pyod_sogaal
    binary_scores_df['pyod_sogaal_dim_redu_binary_cont0.1_lrd0.01_lrg0.01_mom0.3'] = pred
  except:
    print('Exception Raised --> SOGAAL Config 9')

  try:
    clf = pyod.models.deep_svdd.DeepSVDD(verbose=1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd = clf.decision_scores_
    scores_df['pyod_svdd_dim_redu'] = pyod_svdd
    binary_scores_df['pyod_svdd_dim_redu_binary'] = pred
  except:
    print('Exception Raised --> DEEP SVDD Config 1')

  try:
    clf = MO_GAAL(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_dim_redu_cont0.001'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> MOGAAL Config 1')

  try:
    clf = MO_GAAL(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_dim_redu_cont0.01'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> MOGAAL Config 2')

  try:
    clf = MO_GAAL(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_dim_redu_cont0.1'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> MOGAAL Config 3')

  try:
    clf = MO_GAAL(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_dim_redu_cont0.2'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> MOGAAL Config 4')

  try:
    clf = MO_GAAL(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['pyod_mogaal_dim_redu_cont0.5'] = pyod_mogaal
    binary_scores_df['pyod_mogaal_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> MOGAAL Config 5')

  try:
    clf = DeepSVDD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_dim_redu_cont0.001'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_dim_redu_binary_cont0.001'] = pred
  except:
    print('Exception Raised --> DEEPSVDD Config 2')

  try:
    clf = DeepSVDD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_dim_redu_cont0.01'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_dim_redu_binary_cont0.01'] = pred
  except:
    print('Exception Raised --> DEEPSVDD Config 3')

  try:
    clf = DeepSVDD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_dim_redu_cont0.1'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_dim_redu_binary_cont0.1'] = pred
  except:
    print('Exception Raised --> DEEPSVDD Config 4')

  try:
    clf = DeepSVDD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_dim_redu_cont0.2'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_dim_redu_binary_cont0.2'] = pred
  except:
    print('Exception Raised --> DEEPSVDD Config 5')

  try:
    clf = DeepSVDD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['pyod_svdd_decfunc_dim_redu_cont0.5'] = pyod_svdd_decfunc
    binary_scores_df['pyod_svdd_decfunc_dim_redu_binary_cont0.5'] = pred
  except:
    print('Exception Raised --> DEEPSVDD Config 6')

#APPLY FEATURE EXTRACTION PIPELINE ON PRINCIPAL COMPONENTS

In [ ]:
def extract_outlierness_features(data, pynomaly=True, sklearn=True, pyod=True, network=False):
  if(pynomaly==True):
    pynomaly_methods(data, scores_df, binary_scores_df)
  if(sklearn==True):
    sklearn_methods(data, scores_df, binary_scores_df)
  if(pyod==True):
    pyod_methods(data, scores_df, binary_scores_df)
  if(network==True):
    network_methods(data, scores_df, binary_scores_df)

In [ ]:
from sklearn.preprocessing import RobustScaler
import time

print('Outlierness Scores from Principal Components Datasets: ')
start_time = time.time()
extract_outlierness_features(data, pynomaly=True, sklearn=True, pyod=True, network=False)
end = time.time()

#save first feature extraction phase time ellapsed
print("Support Features on Principal Components Extraction Time: %.8s seconds" % (end - start_time))
print('\nNull Values in Outlierness Scores DataSets:')
print('Scores_df: ')
scores_df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
scores_df.loc[:, scores_df.isna().any()]
print('--- --- ---')
print('Binary_Scores_df: ')
binary_scores_df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
binary_scores_df.loc[:, binary_scores_df.isna().any()]
print('--- --- ---')

#normalize outlierness scores
scaler = RobustScaler()
scores_df = pd.DataFrame(scaler.fit_transform(scores_df), columns=scores_df.columns)

scores_df.to_csv(directory + "scores_principal_comp.csv")
binary_scores_df.to_csv(directory + "binary_scores_principal_comp.csv")

#OUTLIERNESS SCORES EXTRACTION ON OUTLIERNESS SCORES

In [ ]:
dataname = 'outlierness_scores.csv'
dataname2 = 'dim_redu_original_data.csv'

data = pd.read_csv(directory+dataname, delimiter=',') #load outlierness scores on original data -> apply anomaly detection on it
data.drop('Unnamed: 0', inplace=True, axis=1)

data2 = pd.read_csv(directory+dataname2, delimiter=',') #load principal components on original data -> apply dimensionality reduction on it
data2.drop('Unnamed: 0', inplace=True, axis=1)

##pynomaly

In [ ]:
scores_df = pd.DataFrame()
binary_scores_df = pd.DataFrame()

In [ ]:
def pynomaly_methods(data, scores_df, binary_scores_df):
  #no parameters to test
  try:
    m = loop.LocalOutlierProbability(data, progress_bar=True).fit()
    scores = m.local_outlier_probabilities
    scores_df['inception_pynomaly_loop_original'] = scores
    binary_scores_df['inception_pynomaly_loop_original_binary'] = scores
    binary_scores_df['inception_pynomaly_loop_original_binary'] = np.where(binary_scores_df['inception_pynomaly_loop_original_binary']>=0.5, 1, 0)
  except:
    print('Exception Raised --> LOP Config 1')

  #parameters to test:
  # -eps/min_samples for DBSCAN
  # -extend/n_neighbors for LOP

  try:
    db = DBSCAN(eps=0.1, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps01_sam10_nei10_e1'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam10_nei10_e1'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam10_nei10_e1'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam10_nei10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 1')

  try:
    db = DBSCAN(eps=0.1, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps01_sam20_nei10_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam20_nei10_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam20_nei10_e2'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam20_nei10_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 2')

  try:  
    db = DBSCAN(eps=0.1, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, n_neighbors=20, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps01_sam20_nei20_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam20_nei20_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam20_nei20_e2'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam20_nei20_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 3')

  try:  
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=10, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps01_sam50_nei10_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei10_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei10_e3'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei10_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 4')

  try: 
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=20, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps01_sam50_nei20_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei20_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei20_e3'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei20_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 5')

  try:  
    db = DBSCAN(eps=0.1, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, n_neighbors=50, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps01_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei50_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei50_e3'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps01_sam50_nei50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 6')

  try:
    db = DBSCAN(eps=0.3, min_samples=10, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=1, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilitiesscores_df['inception_dbscan_scores_original_eps03_sam10_e1'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam10_e1'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam10_e1'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam10_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 7')

  try:
    db = DBSCAN(eps=0.3, min_samples=20, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps03_sam20_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam20_e1'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam20_e1'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam20_e1']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 8')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=2,  cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps03_sam50_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e2'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e2'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e2']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 9')

  try:  
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3,cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e3'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 10')

  try:
    db = DBSCAN(eps=0.3, min_samples=50, n_jobs=-1).fit(data)
    m = loop.LocalOutlierProbability(data, extent=3, cluster_labels=list(db.labels_), progress_bar=True).fit()
    scores_dbscan = m.local_outlier_probabilities
    scores_df['inception_dbscan_scores_original_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e3'] = scores_dbscan
    binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e3'] = np.where(binary_scores_df['inception_dbscan_scores_original_binary_eps03_sam50_e3']>=0.5, 1, 0)
  except:
    print('Exception Raised --> DBSCAN Config 11')

##sklearn algorithms

In [ ]:
def sklearn_methods(data, scores_df, binary_scores_df):
  #Minimum Covariance Determinant
  try:
    ee = EllipticEnvelope(contamination=0.001)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['inception_elliptic_decfunc_original_cont0.001'] = dec_func
    scores_df['inception_elliptic_mahalanobis_original_cont0.001'] = maha
    binary_scores_df['inception_elliptic_env_original_binary_cont0.001'] = pred
    binary_scores_df['inception_elliptic_env_original_binary_cont0.001'] = np.where(binary_scores_df['inception_elliptic_env_original_binary_cont0.001']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 1')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.01)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['inception_elliptic_decfunc_original_cont0.01'] = dec_func
    scores_df['inception_elliptic_mahalanobis_original_cont0.01'] = maha
    binary_scores_df['inception_elliptic_env_original_binary_cont0.01'] = pred
    binary_scores_df['inception_elliptic_env_original_binary_cont0.01'] = np.where(binary_scores_df['inception_elliptic_env_original_binary_cont0.01']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 2')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.1)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['inception_elliptic_decfunc_original_cont0.1'] = dec_func
    scores_df['inception_elliptic_mahalanobis_original_cont0.1'] = maha
    binary_scores_df['inception_elliptic_env_original_binary_cont0.1'] = pred
    binary_scores_df['inception_elliptic_env_original_binary_cont0.1'] = np.where(binary_scores_df['inception_elliptic_env_original_binary_cont0.1']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 3')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.2)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['inception_elliptic_decfunc_original_cont0.2'] = dec_func
    scores_df['inception_elliptic_mahalanobis_original_cont0.2'] = maha
    binary_scores_df['inception_elliptic_env_original_binary_cont0.2'] = pred
    binary_scores_df['inception_elliptic_env_original_binary_cont0.2'] = np.where(binary_scores_df['inception_elliptic_env_original_binary_cont0.2']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 4')

  try:
    #Minimum Covariance Determinant
    ee = EllipticEnvelope(contamination=0.5)
    score = ee.fit_predict(data)
    dec_func = ee.decision_function(data)
    maha = ee.mahalanobis(data)
    pred = ee.predict(data)
    scores_df['inception_elliptic_decfunc_original_cont0.5'] = dec_func
    scores_df['inception_elliptic_mahalanobis_original_cont0.5'] = maha
    binary_scores_df['inception_elliptic_env_original_binary_cont0.5'] = pred
    binary_scores_df['inception_elliptic_env_original_binary_cont0.5'] = np.where(binary_scores_df['inception_elliptic_env_original_binary_cont0.5']==-1, 1,0)
  except:
    print('Exception Raised --> Elliptic Envelope Config 5')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.001, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['inception_iso_forest_paper_score_original_cont0.001'] = original_paper_score
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.001'] = pred
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.001'] = np.where(binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.001']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 1')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.01, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['inception_iso_forest_paper_score_original_cont0.01'] = original_paper_score
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.01'] = pred
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.01'] = np.where(binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.01']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 2')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.1, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['inception_iso_forest_paper_score_original_cont0.1'] = original_paper_score
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.1'] = pred
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.1'] = np.where(binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.1']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 3')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.2, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['inception_iso_forest_paper_score_original_cont0.2'] = original_paper_score
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.2'] = pred
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.2'] = np.where(binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.2']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 4')

  try:
    #isolation forest
    iso = IsolationForest(contamination=0.5, n_jobs=-1)
    #returns -1 for outliers and +1 for inliers
    pred = iso.fit_predict(data)
    #using decision function i can obtain outlierness for each observation
    sklearn_score_anomalies = iso.decision_function(data)
    original_paper_score = [-1*s + 0.5 for s in sklearn_score_anomalies]
    scores_df['inception_iso_forest_paper_score_original_cont0.5'] = original_paper_score
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.5'] = pred
    binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.5'] = np.where(binary_scores_df['inception_iso_forest_paper_score_original_binary_cont0.5']==-1, 1,0)
  except:
    print('Exception Raised --> Iso Forest Config 5')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=10, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['inception_lof_score_original_mink_nei10'] = lof_score
    binary_scores_df['inception_lof_score_original_binary_mink_nei10'] = pred
    binary_scores_df['inception_lof_score_original_binary_mink_nei10'] = np.where(binary_scores_df['inception_lof_score_original_binary_mink_nei10']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 1')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=20, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['inception_lof_score_original_mink_nei20'] = lof_score
    binary_scores_df['inception_lof_score_original_binary_mink_nei20'] = pred
    binary_scores_df['inception_lof_score_original_binary_mink_nei20'] = np.where(binary_scores_df['inception_lof_score_original_binary_mink_nei20']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 2')

  try:
    #Local Outlier Factor
    lof = LocalOutlierFactor(novelty=True, metric='minkowski', n_neighbors=50, n_jobs=-1)
    lof.fit(data)
    lof_score = lof.decision_function(data)
    pred = lof.predict(data)
    scores_df['inception_lof_score_original_mink_nei50'] = lof_score
    binary_scores_df['inception_lof_score_original_binary_mink_nei50'] = pred
    binary_scores_df['inception_lof_score_original_binary_mink_nei50'] = np.where(binary_scores_df['inception_lof_score_original_binary_mink_nei50']==-1, 1,0)
  except:
    print('Exception Raised --> LOF Config 3')

  # from sklearn.svm import OneClassSVM
  # try:
  #   #One Class SVM
  #   ee = OneClassSVM(nu=0.001, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['inception_oneclass_SVM_original_linear_nu0.001'] = ocsvm_score
  #   binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.001'] = pred
  #   binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.001'] = np.where(binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.001']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 1')

  # try:
  #   ee = OneClassSVM(nu=0.01, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['inception_oneclass_SVM_original_linear_nu0.01'] = ocsvm_score
  #   binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.01'] = pred
  #   binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.01'] = np.where(binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.01']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 2')

  # try:
  #   ee = OneClassSVM(nu=0.1, kernel='linear')
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['inception_oneclass_SVM_original_linear_nu0.1'] = ocsvm_score
  #   binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.1'] = pred
  #   binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.1'] = np.where(binary_scores_df['inception_oneclass_SVM_original_binary_linear_nu0.1']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 3')

  # try:
  #   ee = OneClassSVM(nu=0.001)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['inception_oneclass_SVM_original_rbf_nu0.001'] = ocsvm_score
  #   binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.001'] = pred
  #   binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.001'] = np.where(binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.001']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 4')

  # try:  
  #   ee = OneClassSVM(nu=0.01)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['inception_oneclass_SVM_original_rbf_nu0.01'] = ocsvm_score
  #   binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.01'] = pred
  #   binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.01'] = np.where(binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.01']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 5')

  # try:
  #   ee = OneClassSVM(nu=0.1)
  #   pred = ee.fit_predict(data)
  #   ocsvm_score = ee.decision_function(data)
  #   scores_df['inception_oneclass_SVM_original_rbf_nu0.1'] = ocsvm_score
  #   binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.1'] = pred
  #   binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.1'] = np.where(binary_scores_df['inception_oneclass_SVM_original_binary_rbf_nu0.1']==-1, 1,0)
  # except:
  #   print('Exception Raised --> One Class SVM Config 6')

##pyod

In [ ]:
from pyod.models.mcd import MCD
from pyod.models.suod import SUOD
from pyod.models.loda import LODA
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.cblof import CBLOF
from pyod.models.ocsvm import OCSVM
from pyod.models.cof import COF
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
from pyod.models.sos import SOS
from pyod.models.pca import PCA

def pyod_methods(data, scores_df, binary_scores_df):
  try:
    clf = COPOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['inception_pyod_copod_original_cont0.001'] = pyod_copod
    binary_scores_df['inception_pyod_copod_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> COPOD Config 1')

  try:
    clf = COPOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['inception_pyod_copod_original_cont0.01'] = pyod_copod
    binary_scores_df['inception_pyod_copod_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> COPOD Config 2')

  try:
    clf = COPOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['inception_pyod_copod_original_cont0.1'] = pyod_copod
    binary_scores_df['inception_pyod_copod_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> COPOD Config 3')

  try:
    clf = COPOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['inception_pyod_copod_original_cont0.2'] = pyod_copod
    binary_scores_df['inception_pyod_copod_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> COPOD Config 4')

  try:
    clf = COPOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_copod = clf.decision_scores_  # raw outlier scores on the train data
    scores_df['inception_pyod_copod_original_cont0.5'] = pyod_copod
    binary_scores_df['inception_pyod_copod_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> COPOD Config 5')

  try:
    clf = ECOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['inception_pyod_ecod_original_cont0.001'] = pyod_ecod
    binary_scores_df['inception_pyod_ecod_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> ECOD Config 1')

  try:
    clf = ECOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['inception_pyod_ecod_original_cont0.01'] = pyod_ecod
    binary_scores_df['inception_pyod_ecod_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> ECOD Config 2')

  try:
    clf = ECOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['inception_pyod_ecod_original_cont0.1'] = pyod_ecod
    binary_scores_df['inception_pyod_ecod_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> ECOD Config 3')

  try:
    clf = ECOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['inception_pyod_ecod_original_cont0.2'] = pyod_ecod
    binary_scores_df['inception_pyod_ecod_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> ECOD Config 4')

  try:
    clf = ECOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ecod = clf.decision_scores_
    scores_df['inception_pyod_ecod_original_cont0.5'] = pyod_ecod
    binary_scores_df['inception_pyod_ecod_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> ECOD Config 5')

  try:
    clf = SOS(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['inception_pyod_sos_original_cont0.001'] = pyod_sos
    binary_scores_df['inception_pyod_sos_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> SOS Config 1')

  try:
    clf = SOS(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['inception_pyod_sos_original_cont0.01'] = pyod_sos
    binary_scores_df['inception_pyod_sos_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> SOS Config 2')

  try:
    clf = SOS(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['inception_pyod_sos_original_cont0.1'] = pyod_sos
    binary_scores_df['inception_pyod_sos_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> SOS Config 3')

  try:
    clf = SOS(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['inception_pyod_sos_original_cont0.2'] = pyod_sos
    binary_scores_df['inception_pyod_sos_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> SOS Config 4')

  try:
    clf = SOS(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sos = clf.decision_scores_
    scores_df['inception_pyod_sos_original_cont0.5'] = pyod_sos
    binary_scores_df['inception_pyod_sos_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> SOS Config 5')

  try:
    clf = PCA(n_components=2, contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.001_whitFalse'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.001_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 1 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.01_whitFalse'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.01_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 2 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.1_whitFalse'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.1_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 3 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.2_whitFalse'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.2_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 4 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.5_whitFalse'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.5_whitFalse'] = pred
  except:
    print('Exception Raised -> PCA Config 5 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.001_whitTrue'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.001_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 6 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.01_whitTrue'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.01_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 7 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.1_whitTrue'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.1_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 8 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.2_whitTrue'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.2_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 9 data maybe not converged')

  try:
    clf = PCA(n_components=2, contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_pca = clf.decision_scores_
    scores_df['inception_pyod_pca_original_cont0.5_whitTrue'] = pyod_pca
    binary_scores_df['inception_pyod_pca_original_binary_cont0.5_whitTrue'] = pred
  except:
    print('Exception Raised -> PCA Config 10 data maybe not converged')

  try:
    clf = MCD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['inception_pyod_mcd_original_cont0.001'] = pyod_mcd
    binary_scores_df['inception_pyod_mcd_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> MCD Config 1')

  try:
    clf = MCD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['inception_pyod_mcd_original_cont0.01'] = pyod_mcd
    binary_scores_df['inception_pyod_mcd_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> MCD Config 2')

  try:
    clf = MCD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['inception_pyod_mcd_original_cont0.1'] = pyod_mcd
    binary_scores_df['inception_pyod_mcd_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> MCD Config 3')

  try:
    clf = MCD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['inception_pyod_mcd_original_cont0.2'] = pyod_mcd
    binary_scores_df['inception_pyod_mcd_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> MCD Config 4')

  try:
    clf = MCD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mcd = clf.decision_scores_
    scores_df['inception_pyod_mcd_original_cont0.5'] = pyod_mcd
    binary_scores_df['inception_pyod_mcd_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> MCD Config 5')

  try:
    clf = OCSVM(contamination=0.001, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.001_nu0.001'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 1')

  try:
    clf = OCSVM(contamination=0.01, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.01_nu0.001'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.01_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 2')

  try:
    clf = OCSVM(contamination=0.1, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.1_nu0.001'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.1_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 3')

  try:
    clf = OCSVM(contamination=0.2, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.2_nu0.001'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.2_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 4')

  try:
    clf = OCSVM(contamination=0.5, nu=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.5_nu0.001'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.5_nu0.001'] = pred
  except:
    print('Exception Raised -> OCSVM Config 5')

  try:
    clf = OCSVM(contamination=0.001, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.001_nu0.1'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 6')

  try:
    clf = OCSVM(contamination=0.001, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.001_nu0.3'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 7')

  try:
    clf = OCSVM(contamination=0.001, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.001_nu0.5'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 8')

  try:
    clf = OCSVM(contamination=0.01, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.01_nu0.1'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 9')

  try:
    clf = OCSVM(contamination=0.01, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.01_nu0.3'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 10')

  try:
    clf = OCSVM(contamination=0.01, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.01_nu0.5'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.001_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 11')

  try:
    clf = OCSVM(contamination=0.1, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.1_nu0.1'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.1_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 12')

  try:
    clf = OCSVM(contamination=0.1, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.1_nu0.3'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.1_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 13')

  try:
    clf = OCSVM(contamination=0.1, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.1_nu0.5'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.1_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 14')

  try:
    clf = OCSVM(contamination=0.2, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.2_nu0.1'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.2_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 15')

  try:
    clf = OCSVM(contamination=0.2, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.2_nu0.3'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.2_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 16')

  try:
    clf = OCSVM(contamination=0.2, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.2_nu0.5'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.2_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 17')

  try:
    clf = OCSVM(contamination=0.5, nu=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.5_nu0.1'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.5_nu0.1'] = pred
  except:
    print('Exception Raised -> OCSVM Config 18')

  try:
    clf = OCSVM(contamination=0.5, nu=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.5_nu0.3'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.5_nu0.3'] = pred
  except:
    print('Exception Raised -> OCSVM Config 19')

  try:
    clf = OCSVM(contamination=0.5, nu=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_ocsvm = clf.decision_scores_
    scores_df['inception_pyod_ocsvm_original_cont0.5_nu0.5'] = pyod_ocsvm
    binary_scores_df['inception_pyod_ocsvm_original_binary_cont0.5_nu0.5'] = pred
  except:
    print('Exception Raised -> OCSVM Config 20')

  try:
    clf = COF(contamination=0.001, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.001_nei10'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.001_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 1')

  try:
    clf = COF(contamination=0.01, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.01_nei10'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.01_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 2')

  try:
    clf = COF(contamination=0.1, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.1_nei10'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.1_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 3')

  try:
    clf = COF(contamination=0.2, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.2_nei10'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.2_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 4')

  try:
    clf = COF(contamination=0.5, n_neighbors=10)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.5_nei10'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.5_nei10'] = pred
  except:
    print('Exception Raised -> COF Config 5')

  try:
    clf = COF(contamination=0.001, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.001_nei20'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.001_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 6')

  try:
    clf = COF(contamination=0.01, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.01_nei20'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.01_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 7')

  try:
    clf = COF(contamination=0.1, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.1_nei20'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.1_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 8')

  try:
    clf = COF(contamination=0.2, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.2_nei20'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.2_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 9')

  try:
    clf = COF(contamination=0.5, n_neighbors=20)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.5_nei20'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.5_nei20'] = pred
  except:
    print('Exception Raised -> COF Config 10')

  try:
    clf = COF(contamination=0.001, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.001_nei50'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.001_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 11')

  try:
    clf = COF(contamination=0.01, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.01_nei50'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.01_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 12')

  try:
    clf = COF(contamination=0.1, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.1_nei50'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.1_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 13')

  try:
    clf = COF(contamination=0.2, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.2_nei50'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.2_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 14')

  try:
    clf = COF(contamination=0.5, n_neighbors=50)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cof = clf.decision_scores_
    scores_df['inception_pyod_cof_original_cont0.5_nei50'] = pyod_cof
    binary_scores_df['inception_pyod_cof_original_binary_cont0.5_nei50'] = pred
  except:
    print('Exception Raised -> COF Config 15')

  try:
    clf = CBLOF(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['inception_pyod_cblof_original_cont0.001'] = pyod_cblof
    binary_scores_df['inception_pyod_cblof_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> CBLOF Config 1')

  try:  
    clf = CBLOF(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['inception_pyod_cblof_original_cont0.01'] = pyod_cblof
    binary_scores_df['inception_pyod_cblof_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> CBLOF Config 2')

  try:
    clf = CBLOF(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['inception_pyod_cblof_original_cont0.1'] = pyod_cblof
    binary_scores_df['inception_pyod_cblof_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> CBLOF Config 3')

  try:
    clf = CBLOF(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['inception_pyod_cblof_original_cont0.2'] = pyod_cblof
    binary_scores_df['inception_pyod_cblof_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> CBLOF Config 4')

  try:
    clf = CBLOF(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_cblof = clf.decision_scores_
    scores_df['inception_pyod_cblof_original_cont0.5'] = pyod_cblof
    binary_scores_df['inception_pyod_cblof_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> CBLOF Config 5')

  try:
    clf = HBOS(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['inception_pyod_hbos_original_cont0.001'] = pyod_hbos
    binary_scores_df['inception_pyod_hbos_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> HBOS Config 1')

  try:
    clf = HBOS(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['inception_pyod_hbos_original_cont0.01'] = pyod_hbos
    binary_scores_df['inception_pyod_hbos_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> HBOS Config 2')

  try:
    clf = HBOS(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['inception_pyod_hbos_original_cont0.1'] = pyod_hbos
    binary_scores_df['inception_pyod_hbos_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> HBOS Config 3')

  try:
    clf = HBOS(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['inception_pyod_hbos_original_cont0.2'] = pyod_hbos
    binary_scores_df['inception_pyod_hbos_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> HBOS Config 4')

  try:
    clf = HBOS(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_hbos = clf.decision_scores_
    scores_df['inception_pyod_hbos_original_cont0.5'] = pyod_hbos
    binary_scores_df['inception_pyod_hbos_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> HBOS Config 5')

  try:
    clf = KNN(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['inception_pyod_knn_original_cont0.001'] = pyod_knn
    binary_scores_df['inception_pyod_knn_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> KNN Config 1')

  try:
    clf = KNN(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['inception_pyod_knn_original_cont0.01'] = pyod_knn
    binary_scores_df['inception_pyod_knn_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> KNN Config 2')

  try:  
    clf = KNN(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['inception_pyod_knn_original_cont0.1'] = pyod_knn
    binary_scores_df['inception_pyod_knn_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> KNN Config 3')

  try:
    clf = KNN(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['inception_pyod_knn_original_cont0.2'] = pyod_knn
    binary_scores_df['inception_pyod_knn_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> KNN Config 4')

  try:
    clf = KNN(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_knn = clf.decision_scores_
    scores_df['inception_pyod_knn_original_cont0.5'] = pyod_knn
    binary_scores_df['inception_pyod_knn_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> KNN Config 5')

  try:
    clf = FeatureBagging(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['inception_pyod_featbagg_original_cont0.001'] = pyod_featbagg
    binary_scores_df['inception_pyod_featbagg_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 1')

  try:
    clf = FeatureBagging(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['inception_pyod_featbagg_original_cont0.01'] = pyod_featbagg
    binary_scores_df['inception_pyod_featbagg_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 2')

  try:
    clf = FeatureBagging(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['inception_pyod_featbagg_original_cont0.1'] = pyod_featbagg
    binary_scores_df['inception_pyod_featbagg_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 3')

  try:
    clf = FeatureBagging(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['inception_pyod_featbagg_original_cont0.2'] = pyod_featbagg
    binary_scores_df['inception_pyod_featbagg_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 4')

  try:
    clf = FeatureBagging(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_featbagg = clf.decision_scores_
    scores_df['inception_pyod_featbagg_original_cont0.5'] = pyod_featbagg
    binary_scores_df['inception_pyod_featbagg_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> Feature Bagging Config 5')

  try:
    clf = LODA(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['inception_pyod_loda_original_cont0.001'] = pyod_loda
    binary_scores_df['inception_pyod_loda_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> LODA Config 1')

  try:
    clf = LODA(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['inception_pyod_loda_original_cont0.01'] = pyod_loda
    binary_scores_df['inception_pyod_loda_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> LODA Config 2')

  try:
    clf = LODA(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['inception_pyod_loda_original_cont0.1'] = pyod_loda
    binary_scores_df['inception_pyod_loda_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> LODA Config 3')

  try:
    clf = LODA(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['inception_pyod_loda_original_cont0.2'] = pyod_loda
    binary_scores_df['inception_pyod_loda_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> LODA Config 4')

  try:
    clf = LODA(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_loda = clf.decision_scores_
    scores_df['inception_pyod_loda_original_cont0.5'] = pyod_loda
    binary_scores_df['inception_pyod_loda_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> LODA Config 5')

  try:
    clf = SUOD(contamination=0.001, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['inception_pyod_suod_original_cont0.001'] = pyod_suod
    binary_scores_df['inception_pyod_suod_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> SUOD Config 1')

  try:
    clf = SUOD(contamination=0.01, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['inception_pyod_suod_original_cont0.01'] = pyod_suod
    binary_scores_df['inception_pyod_suod_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> SUOD Config 2')

  try:
    clf = SUOD(contamination=0.1, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['inception_pyod_suod_original_cont0.1'] = pyod_suod
    binary_scores_df['inception_pyod_suod_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> SUOD Config 3')

  try:
    clf = SUOD(contamination=0.2, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['inception_pyod_suod_original_cont0.2'] = pyod_suod
    binary_scores_df['inception_pyod_suod_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> SUOD Config 4')

  try:
    clf = SUOD(contamination=0.5, n_jobs=-1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_suod = clf.decision_scores_
    scores_df['inception_pyod_suod_original_cont0.5'] = pyod_suod
    binary_scores_df['inception_pyod_suod_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> SUOD Config 5')

##network methods

In [ ]:
#autoencoder method - (improve network)
from pyod.models.auto_encoder import AutoEncoder
def network_methods(data):
  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['inception_pyod_encoder_original_cont0.001'] = pyod_encoder
    binary_scores_df['inception_pyod_encoder_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['inception_pyod_encoder_original_cont0.01'] = pyod_encoder
    binary_scores_df['inception_pyod_encoder_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['inception_pyod_encoder_original_cont0.1'] = pyod_encoder
    binary_scores_df['inception_pyod_encoder_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['inception_pyod_encoder_original_cont0.2'] = pyod_encoder
    binary_scores_df['inception_pyod_encoder_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  try:
    clf = AutoEncoder(epochs=200, hidden_neurons=[data.shape[1], data.shape[1]/2,data.shape[1]/2,data.shape[1]], contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_encoder = clf.decision_scores_
    scores_df['inception_pyod_encoder_original_cont0.5'] = pyod_encoder
    binary_scores_df['inception_pyod_encoder_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> Auto Encoder Config')

  #vae method
  from pyod.models.vae import VAE

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['inception_pyod_vae_original_cont0.001'] = pyod_vae
    binary_scores_df['inception_pyod_vae_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['inception_pyod_vae_original_cont0.01'] = pyod_vae
    binary_scores_df['inception_pyod_vae_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['inception_pyod_vae_original_cont0.1'] = pyod_vae
    binary_scores_df['inception_pyod_vae_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['inception_pyod_vae_original_cont0.2'] = pyod_vae
    binary_scores_df['inception_pyod_vae_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> VAE Config')

  try:
    clf = VAE(epochs=200, encoder_neurons=[data.shape[1], data.shape[1]/2], decoder_neurons=[data.shape[1]/2,data.shape[1]], contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_vae = clf.decision_scores_
    scores_df['inception_pyod_vae_original_cont0.5'] = pyod_vae
    binary_scores_df['inception_pyod_vae_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> VAE Config')

  #so_gaal method
  from pyod.models.so_gaal import SO_GAAL

  try:
    clf = SO_GAAL(contamination=0.001, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.001_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.001_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.01, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.01_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.01_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.1_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.1_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.2, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.2_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.2_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.5, lr_d=0.0001 , lr_g=0.0001 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.5_lrd0.0001_lrg0.0001_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.5_lrd0.0001_lrg0.0001_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.1_lrd0.01_lrg0.01_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.1_lrd0.01_lrg0.01_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.3 , lr_g=0.3 ,momentum=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.1_lrd0.3_lrg0.3_mom0.01'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.1_lrd0.3_lrg0.3_mom0.01'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.1_lrd0.01_lrg0.01_mom0.1'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.1_lrd0.01_lrg0.01_mom0.1'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  try:
    clf = SO_GAAL(contamination=0.1, lr_d=0.01 , lr_g=0.01 ,momentum=0.3)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_sogaal = clf.decision_scores_
    scores_df['inception_pyod_sogaal_original_cont0.1_lrd0.01_lrg0.01_mom0.3'] = pyod_sogaal
    binary_scores_df['inception_pyod_sogaal_original_binary_cont0.1_lrd0.01_lrg0.01_mom0.3'] = pred
  except:
    print('Exception Raised -> SO GAAL Config')

  #deepsvdd method
  import pyod.models.deep_svdd
  try:
    clf = pyod.models.deep_svdd.DeepSVDD(verbose=1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd = clf.decision_scores_
    scores_df['inception_pyod_svdd_original'] = pyod_svdd
    binary_scores_df['inception_pyod_svdd_original_binary'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  #mo_gaal method
  from pyod.models.mo_gaal import MO_GAAL

  try:
    clf = MO_GAAL(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['inception_pyod_mogaal_original_cont0.001'] = pyod_mogaal
    binary_scores_df['inception_pyod_mogaal_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['inception_pyod_mogaal_original_cont0.01'] = pyod_mogaal
    binary_scores_df['inception_pyod_mogaal_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['inception_pyod_mogaal_original_cont0.1'] = pyod_mogaal
    binary_scores_df['inception_pyod_mogaal_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['inception_pyod_mogaal_original_cont0.2'] = pyod_mogaal
    binary_scores_df['inception_pyod_mogaal_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  try:
    clf = MO_GAAL(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_mogaal = clf.decision_scores_
    scores_df['inception_pyod_mogaal_original_cont0.5'] = pyod_mogaal
    binary_scores_df['inception_pyod_mogaal_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> MO GAAL Config')

  #deep_svd method
  from pyod.models.deep_svdd import DeepSVDD

  try:
    clf = DeepSVDD(contamination=0.001)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['inception_pyod_svdd_decfunc_original_cont0.001'] = pyod_svdd_decfunc
    binary_scores_df['inception_pyod_svdd_decfunc_original_binary_cont0.001'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.01)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['inception_pyod_svdd_decfunc_original_cont0.01'] = pyod_svdd_decfunc
    binary_scores_df['inception_pyod_svdd_decfunc_original_binary_cont0.01'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.1)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['inception_pyod_svdd_decfunc_original_cont0.1'] = pyod_svdd_decfunc
    binary_scores_df['inception_pyod_svdd_decfunc_original_binary_cont0.1'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.2)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['inception_pyod_svdd_decfunc_original_cont0.2'] = pyod_svdd_decfunc
    binary_scores_df['inception_pyod_svdd_decfunc_original_binary_cont0.2'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

  try:
    clf = DeepSVDD(contamination=0.5)
    clf.fit(data)
    pred = clf.predict(data)
    pyod_svdd_decfunc = clf.decision_scores_
    scores_df['inception_pyod_svdd_decfunc_original_cont0.5'] = pyod_svdd_decfunc
    binary_scores_df['inception_pyod_svdd_decfunc_original_binary_cont0.5'] = pred
  except:
    print('Exception Raised -> DEEP SVDD Config')

#DIMENSIONALITY REDUCTION TECHNIQUES ON PROJECTED FEATURES


In [ ]:
def sammon(x, n, display = 0, inputdist = 'raw', maxhalves = 20, maxiter = 1, tolfun = 1e-9, init = 'default'):

    import numpy as np 
    from scipy.spatial.distance import cdist

    """Perform Sammon mapping on dataset x
    y = sammon(x) applies the Sammon nonlinear mapping procedure on
    multivariate data x, where each row represents a pattern and each column
    represents a feature.  On completion, y contains the corresponding
    co-ordinates of each point on the map.  By default, a two-dimensional
    map is created.  Note if x contains any duplicated rows, SAMMON will
    fail (ungracefully). 
    [y,E] = sammon(x) also returns the value of the cost function in E (i.e.
    the ess of the mapping).
    An N-dimensional output map is generated by y = sammon(x,n) .
    A set of optimisation options can be specified using optional
    arguments, y = sammon(x,n,[OPTS]):
       maxiter        - maximum number of iterations
       tolfun         - relative tolerance on objective function
       maxhalves      - maximum number of step halvings
       input          - {'raw','distance'} if set to 'distance', X is 
                        interpreted as a matrix of pairwise distances.
       display        - 0 to 2. 0 least verbose, 2 max verbose.
       init           - {'pca', 'cmdscale', random', 'default'}
                        default is 'pca' if input is 'raw', 
                        'msdcale' if input is 'distance'
    The default options are retrieved by calling sammon(x) with no
    parameters.
    File        : sammon.py
    Date        : 18 April 2014
    Authors     : Tom J. Pollard (tom.pollard.11@ucl.ac.uk)
                : Ported from MATLAB implementation by 
                  Gavin C. Cawley and Nicola L. C. Talbot
    Description : Simple python implementation of Sammon's non-linear
                  mapping algorithm [1].
    References  : [1] Sammon, John W. Jr., "A Nonlinear Mapping for Data
                  Structure Analysis", IEEE Transactions on Computers,
                  vol. C-18, no. 5, pp 401-409, May 1969.
    Copyright   : (c) Dr Gavin C. Cawley, November 2007.
    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program; if not, write to the Free Software
    Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
    """

    # Create distance matrix unless given by parameters
    if inputdist == 'distance':
        D = x
        if init == 'default':
            init = 'cmdscale'
    else:
        D = cdist(x, x)
        if init == 'default':
            init = 'pca'

    if inputdist == 'distance' and init == 'pca':
        raise ValueError("Cannot use init == 'pca' when inputdist == 'distance'")

    if np.count_nonzero(np.diagonal(D)) > 0:
        raise ValueError("The diagonal of the dissimilarity matrix must be zero")

    # Remaining initialisation
    N = x.shape[0]
    scale = 0.5 / D.sum()
    D = D + np.eye(N)     

    if np.count_nonzero(D<=0) > 0:
        raise ValueError("Off-diagonal dissimilarities must be strictly positive")   

    Dinv = 1 / D
    if init == 'pca':
        [UU,DD,_] = np.linalg.svd(x)
        y = UU[:,:n]*DD[:n] 
    elif init == 'cmdscale':
        from cmdscale import cmdscale
        y,e = cmdscale(D)
        y = y[:,:n]
    else:
        y = np.random.normal(0.0,1.0,[N,n])
    one = np.ones([N,n])
    d = cdist(y,y) + np.eye(N)
    dinv = 1. / d
    delta = D-d 
    E = ((delta**2)*Dinv).sum() 

    # Get on with it
    for i in range(maxiter):
        # Compute gradient, Hessian and search direction (note it is actually
        # 1/4 of the gradient and Hessian, but the step size is just the ratio
        # of the gradient and the diagonal of the Hessian so it doesn't
        # matter).
        delta = dinv - Dinv
        deltaone = np.dot(delta,one)
        g = np.dot(delta,y) - (y * deltaone)
        dinv3 = dinv ** 3
        y2 = y ** 2
        H = np.dot(dinv3,y2) - deltaone - np.dot(2,y) * np.dot(dinv3,y) + y2 * np.dot(dinv3,one)
        s = -g.flatten(order='F') / np.abs(H.flatten(order='F'))
        y_old    = y

        # Use step-halving procedure to ensure progress is made
        for j in range(maxhalves):
            s_reshape = np.reshape(s, (-1,n),order='F')
            y = y_old + s_reshape
            d = cdist(y, y) + np.eye(N)
            dinv = 1 / d
            delta = D - d
            E_new = ((delta**2)*Dinv).sum()
            if E_new < E:
                break
            else:
                s = 0.5*s

        # Bomb out if too many halving steps are required
        if j == maxhalves-1:
            print('Warning: maxhalves exceeded. Sammon mapping may not converge...')

        # Evaluate termination criterion
        if abs((E - E_new) / E) < tolfun:
            if display:
                print('TolFun exceeded: Optimisation terminated')
            break

        # Report progress
        E = E_new
        if display > 1:
            print('epoch = %d : E = %12.10f'% (i+1, E * scale))

    if i == maxiter-1:
        print('Warning: maxiter exceeded. Sammon mapping may not have converged...')

    # Fiddle stress to match the original Sammon paper
    E = E * scale
    
    return [y,E]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import random_projection
from sklearn.manifold import Isomap
from keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.decomposition import KernelPCA
import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import manifold
from functools import partial
from collections import OrderedDict
from time import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter

#create new dataframes. they will contain principal components extracted from different dimensionality reduction techniques
#df_dimredu contains components extracted from outlierness scores
#df_dataredu contains components extracted from original data

df_dimredu = pd.DataFrame()
df_dataredu = pd.DataFrame()
df_binredu = pd.DataFrame()

def dim_redu_methods(data, scores_df, binary_scores_df):
  pca = PCA(n_components=2)
  out_pca = pd.DataFrame()
  out_pca_scores = pd.DataFrame()
  out_pca_bin = pd.DataFrame()
  try:
    pca.fit(data)
    out_pca = pd.DataFrame(pca.transform(data), index=data.index)
  except:
    print('Exception Raised --> PCA Config 1')

  try:
    pca.fit(scores_df)
    out_pca_scores = pd.DataFrame(pca.transform(scores_df), index=scores_df.index)
  except:
    print('Exception Raised --> PCA Config 2')

  try:
    pca.fit(binary_scores_df)
    out_pca_bin = pd.DataFrame(pca.transform(binary_scores_df), index=binary_scores_df.index)
  except:
    print('Exception Raised --> PCA Config 3')

  try:
    df_dataredu['inception_pca1_data'] = out_pca[0]
    df_dataredu['inception_pca2_data'] = out_pca[1]
    df_dimredu['inception_pca1_scores'] = out_pca_scores[0]
    df_dimredu['inception_pca2_scores'] = out_pca_scores[1]
    df_binredu['inception_pca1_binary'] = out_pca_bin[0]
    df_binredu['inception_pca2_binary'] = out_pca_bin[1]
  except:
    print('Exception Raised --> PCA Config loading')

  rsp = random_projection.GaussianRandomProjection(n_components=2)
  try:
    ran_proj = rsp.fit_transform(scores_df)
    df_dimredu['inception_sub_proj1_scores'] = ran_proj[:,0]
    df_dimredu['inception_sub_proj2_scores'] = ran_proj[:,1]
  except:
    print('Exception Raised --> Random Projection Config scores')

  try:  
    ran_proj_data = rsp.fit_transform(data)
    df_dataredu['inception_sub_proj1_data'] = ran_proj_data[:,0]
    df_dataredu['inception_sub_proj2_data'] = ran_proj_data[:,1]
  except:
    print('Exception Raised --> Random Projection Config data')

  try:
    ran_proj_bin = rsp.fit_transform(binary_scores_df)
    df_binredu['inception_sub_proj1_binary'] = ran_proj_bin[:,0]
    df_binredu['inception_sub_proj2_binary'] = ran_proj_bin[:,1]
  except:
    print('Exception Raised --> Random Projection Config binary')

  iso = Isomap(n_components=2)

  try:
    iso_proj = iso.fit_transform(scores_df)
    df_dimredu['inception_iso_map_1_scores'] = iso_proj[:,0]
    df_dimredu['inception_iso_map_2_scores'] = iso_proj[:,1]
  except:
    print('Exception Raised --> Iso Map Config scores')

  try:
    iso_proj_data = iso.fit_transform(data)
    df_dataredu['inception_iso_map_1_data'] = iso_proj_data[:,0]
    df_dataredu['inception_iso_map_2_data'] = iso_proj_data[:,1]
  except:
    print('Exception Raised --> Iso Map Config data')

  try:  
    iso_proj_bin = iso.fit_transform(binary_scores_df)
    df_binredu['inception_iso_map_1_binary'] = iso_proj_bin[:,0]
    df_binredu['inception_iso_map_2_binary'] = iso_proj_bin[:,1]
  except:
    print('Exception Raised --> Iso Map Config binary')

  tsne = TSNE(n_components=2)

  try:
    sne_proj = tsne.fit_transform(scores_df)
    df_dimredu['inception_sne_1_scores'] = sne_proj[:,0]
    df_dimredu['inception_sne_2_scores'] = sne_proj[:,1]
  except:
    print('Exception Raised --> TSNE Config scores')

  try:  
    sne_proj_data = tsne.fit_transform(data)
    df_dataredu['inception_sne_1_data'] = sne_proj_data[:,0]
    df_dataredu['inception_sne_2_data'] = sne_proj_data[:,1]
  except:
    print('Exception Raised --> TSNE Config data')

  try:
    sne_proj_bin = tsne.fit_transform(binary_scores_df)
    df_binredu['inception_sne_1_binary'] = sne_proj_bin[:,0]
    df_binredu['inception_sne_2_binary'] = sne_proj_bin[:,1]
  except:
    print('Exception Raised --> TSNE Config binary')

  clf = LinearDiscriminantAnalysis()

  try:
    clf.fit(scores_df, target)
    lda_proj = clf.transform(scores_df)
    df_dimredu['inception_lda_1_scores_0'] = lda_proj[0:lda_proj.shape[0], 0]
    df_dimredu['inception_lda_1_scores_1'] = lda_proj[0:lda_proj.shape[0], 1]
    df_dimredu['inception_lda_1_scores_2'] = lda_proj[0:lda_proj.shape[0], 2]
    df_dimredu['inception_lda_1_scores_3'] = lda_proj[0:lda_proj.shape[0], 3]
    df_dimredu['inception_lda_1_scores_4'] = lda_proj[0:lda_proj.shape[0], 4]
    df_dimredu['inception_lda_1_scores_5'] = lda_proj[0:lda_proj.shape[0], 5]
    df_dimredu['inception_lda_1_scores_6'] = lda_proj[0:lda_proj.shape[0], 6]
    df_dimredu['inception_lda_1_scores_7'] = lda_proj[0:lda_proj.shape[0], 7]
    df_dimredu['inception_lda_1_scores_8'] = lda_proj[0:lda_proj.shape[0], 8]
    df_dimredu['inception_lda_1_scores_9'] = lda_proj[0:lda_proj.shape[0], 9]
    df_dimredu['inception_lda_1_scores_10'] = lda_proj[0:lda_proj.shape[0], 10]
  except:
    print('Exception Raised --> Linear Discriminant Analysis (classes exceeded)')

  try:
    clf.fit(data, target)
    lda_proj_data = clf.transform(data)
    df_dataredu['inception_lda_1_data_0'] = lda_proj_data[0:lda_proj_data.shape[0], 0]
    df_dataredu['inception_lda_1_data_1'] = lda_proj_data[0:lda_proj_data.shape[0], 1]
    df_dataredu['inception_lda_1_data_2'] = lda_proj_data[0:lda_proj_data.shape[0], 2]
    df_dataredu['inception_lda_1_data_3'] = lda_proj_data[0:lda_proj_data.shape[0], 3]
    df_dataredu['inception_lda_1_data_4'] = lda_proj_data[0:lda_proj_data.shape[0], 4]
    df_dataredu['inception_lda_1_data_5'] = lda_proj_data[0:lda_proj_data.shape[0], 5]
    df_dataredu['inception_lda_1_data_6'] = lda_proj_data[0:lda_proj_data.shape[0], 6]
    df_dataredu['inception_lda_1_data_7'] = lda_proj_data[0:lda_proj_data.shape[0], 7]
    df_dataredu['inception_lda_1_data_8'] = lda_proj_data[0:lda_proj_data.shape[0], 8]
    df_dataredu['inception_lda_1_data_9'] = lda_proj_data[0:lda_proj_data.shape[0], 9]
    df_dataredu['inception_lda_1_data_10'] = lda_proj_data[0:lda_proj_data.shape[0], 10]
  except:
    print('Exception Raised --> Linear Discriminant Analysis Config (classes exceeded)')

  try:
    clf.fit(binary_scores_df, target)
    lda_proj_bin = clf.transform(binary_scores_df) 
    df_binredu['inception_lda_1_binary_0'] = lda_proj_bin[0:lda_proj_bin.shape[0], 0]
    df_binredu['inception_lda_1_binary_1'] = lda_proj_bin[0:lda_proj_bin.shape[0], 1]
    df_binredu['inception_lda_1_binary_2'] = lda_proj_bin[0:lda_proj_bin.shape[0], 2]
    df_binredu['inception_lda_1_binary_3'] = lda_proj_bin[0:lda_proj_bin.shape[0], 3]
    df_binredu['inception_lda_1_binary_4'] = lda_proj_bin[0:lda_proj_bin.shape[0], 4]
    df_binredu['inception_lda_1_binary_5'] = lda_proj_bin[0:lda_proj_bin.shape[0], 5]
    df_binredu['inception_lda_1_binary_6'] = lda_proj_bin[0:lda_proj_bin.shape[0], 6]
    df_binredu['inception_lda_1_binary_7'] = lda_proj_bin[0:lda_proj_bin.shape[0], 7]
    df_binredu['inception_lda_1_binary_8'] = lda_proj_bin[0:lda_proj_bin.shape[0], 8]
    df_binredu['inception_lda_1_binary_9'] = lda_proj_bin[0:lda_proj_bin.shape[0], 9]
    df_binredu['inception_lda_1_binary_10'] = lda_proj_bin[0:lda_proj_bin.shape[0], 10]
  except:
    print('Exception Raised --> Linear Discriminant Analysis Config (classes exceeded)')

  kpca = KernelPCA(n_components=2, kernel='rbf', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['inception_rbf_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['inception_rbf_pca_2_scores'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 1')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['inception_rbf_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['inception_rbf_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 1')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['inception_rbf_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['inception_rbf_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 1')


  kpca = KernelPCA(n_components=2, kernel='cosine', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['inception_cosine_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['inception_cosine_pca_2_scores'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 2')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['inception_cosine_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['inception_cosine_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 2')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['inception_cosine_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['inception_cosine_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 2')

  kpca = KernelPCA(n_components=2, kernel='sigmoid', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['inception_sigmoid_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['inception_sigmoid_pca_2_scores'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 3')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['inception_sigmoid_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['inception_sigmoid_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 3')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['inception_sigmoid_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['inception_sigmoid_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 3')

  kpca = KernelPCA(n_components=2, kernel='poly', 
                  gamma=15, random_state=42)

  try:
    X_kpca = kpca.fit_transform(scores_df)
    df_dimredu['inception_poly_pca_1_scores'] = X_kpca[:,0]
    df_dimredu['inception_poly_pca_2_scores_'] = X_kpca[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config scores 4')

  try:
    X_kpca_data = kpca.fit_transform(data)
    df_dataredu['inception_poly_pca_1_data'] = X_kpca_data[:,0]
    df_dataredu['inception_poly_pca_2_data'] = X_kpca_data[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config data 4')

  try:
    X_kpca_bin = kpca.fit_transform(binary_scores_df)
    df_binredu['inception_poly_pca_1_binary'] = X_kpca_bin[:,0]
    df_binredu['inception_poly_pca_2_binary'] = X_kpca_bin[:,1]
  except:
    print('Exception Raised --> Kernel PCA Config binary 4')

  #convert two scores in numeric, to perform sammon function
  if 'inception_pynomaly_loop_original' in scores_df.columns:
    scores_df["inception_pynomaly_loop_original"] = pd.to_numeric(scores_df["inception_pynomaly_loop_original"])
  #scores_df["dbscan_scores_original"] = pd.to_numeric(scores_df["dbscan_scores_original"])

  try:
    # By default, sammon returns a 2-dim array and the error E
    [y, E] = sammon(data, maxiter=20, n=2)
    [y2, E2] = sammon(scores_df, maxiter=20, n=2)

    df_dataredu['inception_sammon_1_data'] = y[:,0]
    df_dataredu['inception_sammon_2_data'] = y[:,1]
    df_dimredu['inception_sammon_1_scores'] = y2[:,0]
    df_dimredu['inception_sammon_2_scores'] = y2[:,1]
  except:
    print('Exception Raised --> Sammon Mapping Config')

  try:
    Axes3D
    n_neighbors = 100
    n_components = 2
    # Set-up manifold methods
    LLE = partial(manifold.LocallyLinearEmbedding, n_neighbors=n_neighbors, n_components=n_components, eigen_solver="auto")
    methods = OrderedDict()
    methods["LLE"] = LLE(method="standard")
    methods["Hessian LLE"] = LLE(method="hessian", eigen_solver='dense')
    methods["Modified LLE"] = LLE(method="modified", eigen_solver='dense')
    methods["MDS"] = manifold.MDS(n_components, max_iter=100, n_init=1)
    methods["SE"] = manifold.SpectralEmbedding(n_components=n_components, n_neighbors=n_neighbors)
  except:
    print('Exception Raised -> MainFold Methods Inizialization')
  try:
    for i, (label, method) in enumerate(methods.items()):
      Y = method.fit_transform(scores_df)
      Y2 = method.fit_transform(data)
      str1 = label+"_1inception_"
      str2 = label+"_2inception_"  
      
      if(label!='Modified LLE'):
        Y3 = method.fit_transform(binary_scores_df)
        df_binredu[str1+"_binary"] = Y3[:,0]
        df_binredu[str2+"_binary"] = Y3[:,1]      
      
      df_dimredu[str1+"_scores"] = Y[:,0]
      df_dimredu[str2+"_scores"] = Y[:,1]
      df_dataredu[str1+"_data"] = Y2[:,0]
      df_dataredu[str2+"_data"] = Y2[:,1]
      print("%s Completed" % (label))
  except:
    print('Exception Raised --> MainFold Methods Calculus')

  try:
    # Fixed dimensions
    input_dim = data.shape[1]
    encoding_dim = 2
    # Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
    input_layer = Input(shape=(input_dim, ))
    encoder_layer_1 = Dense(18, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(9, activation="tanh")(encoder_layer_1)
    encoder_layer_3 = Dense(7, activation="tanh")(encoder_layer_2)
    encoder_layer_4 = Dense(5, activation="relu")(encoder_layer_3)
    encoder_layer_5 = Dense(3, activation="relu")(encoder_layer_4)
    encoder_layer_6 = Dense(encoding_dim, activation="sigmoid")(encoder_layer_5)

    # Crear encoder model
    encoder = Model(inputs=input_layer, outputs=encoder_layer_6)
    # Use the model to predict the factors which sum up the information of interest rates.
    encoded_data = pd.DataFrame(encoder.predict(data))
    encoded_data.columns = ['factor_1', 'factor_2']

    # Fixed dimensions
    input_dim = scores_df.shape[1]
    encoding_dim = 2
    # Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
    input_layer = Input(shape=(input_dim, ))
    encoder_layer_1 = Dense(18, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(9, activation="relu")(encoder_layer_1)
    encoder_layer_3 = Dense(7, activation="relu")(encoder_layer_2)
    encoder_layer_4 = Dense(5, activation="relu")(encoder_layer_3)
    encoder_layer_5 = Dense(3, activation="relu")(encoder_layer_4)
    encoder_layer_6 = Dense(encoding_dim, activation="sigmoid")(encoder_layer_5)

    # Crear encoder model
    encoder = Model(inputs=input_layer, outputs=encoder_layer_6)
    # Use the model to predict the factors which sum up the information of interest rates.
    encoded_scores = pd.DataFrame(encoder.predict(scores_df))
    encoded_scores.columns = ['factor_1', 'factor_2']

    # Fixed dimensions
    input_dim = binary_scores_df.shape[1]
    encoding_dim = 2
    # Number of neurons in each Layer [8, 6, 4, 3, ...] of encoders
    input_layer = Input(shape=(input_dim, ))
    encoder_layer_1 = Dense(18, activation="relu", activity_regularizer=regularizers.l1(10e-5))(input_layer)
    encoder_layer_2 = Dense(9, activation="relu")(encoder_layer_1)
    encoder_layer_3 = Dense(7, activation="relu")(encoder_layer_2)
    encoder_layer_4 = Dense(5, activation="relu")(encoder_layer_3)
    encoder_layer_5 = Dense(3, activation="relu")(encoder_layer_4)
    encoder_layer_6 = Dense(encoding_dim, activation="sigmoid")(encoder_layer_5)

    # Crear encoder model
    encoder = Model(inputs=input_layer, outputs=encoder_layer_6)
    # Use the model to predict the factors which sum up the information of interest rates.
    encoded_bin = pd.DataFrame(encoder.predict(binary_scores_df))
    encoded_bin.columns = ['factor_1', 'factor_2']

    df_dataredu['inception_autoencoder_1_data'] = encoded_data['factor_1']
    df_dataredu['inception_autoencoder_2_data'] = encoded_data['factor_2']
    df_dimredu['inception_autoencoder_1_scores'] = encoded_scores['factor_1']
    df_dimredu['inception_autoencoder_2_scores'] = encoded_scores['factor_2']
    df_binredu['inception_autoencoder_1_binary'] = encoded_bin['factor_1']
    df_binredu['inception_autoencoder_2_binary'] = encoded_bin['factor_2']
  except:
    print('Exception Raised --> AutoEncoder Config')

#APPLY FEATURE EXTRACTION PIPELINE ON PROJECTED FEATURES

In [ ]:
def extract_outlierness_features(data, pynomaly=True, sklearn=True, pyod=True, network=True):
  if(pynomaly==True):
    pynomaly_methods(data, scores_df, binary_scores_df)
  if(sklearn==True):
    sklearn_methods(data, scores_df, binary_scores_df)
  if(pyod==True):
    pyod_methods(data, scores_df, binary_scores_df)
  if(network==True):
    network_methods(data)

In [ ]:
import time

start_time = time.time()

print('Outlierness Scores Extraction: \n')
extract_outlierness_features(data, pynomaly=True,sklearn=True,pyod=True,network=False) #anomaly detection methods applied on outlierness scores

#normalize outlierness scores
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scores_df = pd.DataFrame(scaler.fit_transform(scores_df), columns=scores_df.columns)

print('\nNull Values in Outlierness Scores DataSets:')
print('Scores_df: ')
scores_df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
scores_df.loc[:, scores_df.isna().any()]
print('--- --- ---')
print('Binary_Scores_df: ')
binary_scores_df.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
binary_scores_df.loc[:, binary_scores_df.isna().any()]
print('--- --- ---')

#save outlierness numeric and binary scores to .csv
scores_df.to_csv(directory + "inception_outlierness_scores.csv")
binary_scores_df.to_csv(directory + "inception_binary_scores.csv")

#apply dimensionality reduction methods on principal components
print('\n--- --- --- --- ---\nDimensionality Reduction Features Extraction: ')


dim_redu_methods(data2, scores_df, binary_scores_df)

print('\nNull Values in Dimensionality Reduction DataSets:')
df_dataredu.dropna(axis=1, how="any", thresh=None, subset=None, inplace=True)
print(df_dataredu.isnull().sum())
print('--- --- ---')

#normalize calculated principal components
scaler = RobustScaler()
df_dataredu = pd.DataFrame(scaler.fit_transform(df_dataredu), columns=df_dataredu.columns)

#save dimensionality reduction (applied on principal components) to .csv
df_dataredu.to_csv(directory + "inception_dim_redu_original_data.csv")

end = time.time()
#save first feature extraction phase time ellapsed
print("Support Features Extraction Time: %.8s seconds" % (end - start_time))